In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import optuna
import matplotlib.pyplot as plt

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv')
sub_df = pd.read_csv('/kaggle/input/playground-series-s5e8/sample_submission.csv')

# Feature Engineering - experiments with new features

In [ ]:
# we can create separate column with flag for -1 value
# train_df['no_previous_contact'] = (train_df['pdays'] == -1).astype(int)
# test_df['no_previous_contact'] = (test_df['pdays'] == -1).astype(int)

# We can create additional column with pdays only without -1 values
# train_df['pdays_cleaned'] = train_df['pdays'].where(train_df['pdays'] != -1, np.nan) 
# test_df['pdays_cleaned'] = test_df['pdays'].where(test_df['pdays'] != -1, np.nan) 

# We can create additional column with numeric months
#train_df['month_as_num'] = train_df['month'].map({'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11, 'dec':12})
#test_df['month_as_num'] = test_df['month'].map({'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11, 'dec':12})

In [ ]:
def stability(x):
    if x=='management' or x=='technician' or x=='admin.' or x=='services':
        return 0
    elif x=='blue-collar' or x=='self-employed' or x=='entrepreneur':
        return 1
    else:
        return 2

train_df['job_stability'] = train_df['job'].apply(stability)

In [ ]:
def stability(x):
    #High
    if x=='management' or x=='entrepreneur' or x=='self-employed':
        return 0
    # Middle
    elif x=='technician' or x=='admin.' or x=='services':
        return 1
    # Low
    elif x=='blue-collar' or x=='housemaid':
        return 2
    # No earnings
    else:
        return 3

train_df['job_earnings'] = train_df['job'].apply(stability)

In [ ]:
def stability(x):
    #Not working
    if x=='retired' or x=='student' or x=='unemployed' or x=='unknown': 
        return 0
    # working
    else:
        return 1

train_df['job_is_working'] = train_df['job'].apply(stability)

In [ ]:
train_df['prev_success']=(train_df['poutcome'] == 'success').astype(int)

In [ ]:
train_df['duration_mult_age'] = train_df['duration'] * train_df['age']

In [ ]:
train_df['job_and_education'] = train_df['education'] + "_" + train_df['job']

In [ ]:
train_df['contact_plus_poutcome'] = train_df['contact'] + ' ' + train_df['poutcome']

In [ ]:
def stability(x):
    if x < 30: 
        return 0
    elif x >= 30 and x <= 60:
        return 1
    else:
        return 2

train_df['age_group'] = train_df['age'].apply(stability)

In [ ]:
def month(x):
    if x < 5:
        return 0
    elif x >= 5 and x <=8:
        return 1
    else:
        return 2 

train_df['month_group'] = train_df['month_as_num'].apply(month)

In [ ]:
def duration(x):
    # short
    if x < 120:
        return 0
    # medium
    elif x >= 120 and x <=250:
        return 1
    # long
    else:
        return 2

train_df['duration_long'] = train_df['duration'].apply(duration)

In [ ]:
mapping = ({'yes':1,"no":0})
train_df['loan_code'] = train_df['loan'].map(mapping)
train_df['housing_code'] = train_df['housing'].map(mapping)

train_df['loan_plus_housing'] = (train_df['loan_code']+train_df['housing_code']).astype(int)

In [ ]:
def add_no_and_unknown_counts(df):
    # Zliczanie 'NO' w kolumnach A, B, C
    df['no_count'] = df[['default', 'housing', 'loan']].eq('no').sum(axis=1)
    
    # Zliczanie 'UNKNOWN' w kolumnach D, E
    df['unknown_count'] = df[['education', 'contact','poutcome']].eq('unknown').sum(axis=1)
    
    return df

add_no_and_unknown_counts(train_df)


In [ ]:
# train_df['day'] = train_df['day'].astype(str)

In [ ]:
train_df.columns

In [ ]:
# Check if new column can influence model learning process
train_df['month_as_num'] = train_df['month'].map({'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11, 'dec':12})
DAYS_IN_YEAR = 31 * 12  
# Number of the day in whole year
train_df['day_of_year'] = (train_df['month_as_num'] - 1) * 31 + train_df['day']
# Coding as cyclic data
train_df['day_of_year_sin'] = np.sin(2 * np.pi * train_df['day_of_year'] / DAYS_IN_YEAR)
train_df['day_of_year_cos'] = np.cos(2 * np.pi * train_df['day_of_year'] / DAYS_IN_YEAR)

train_df['balance_log'] = train_df['balance'].where(train_df['balance'] > -1, 0).apply(np.log1p)
train_df['duration_log'] = train_df['duration'].where(train_df['duration'] > -1, 0).apply(np.log1p)

# ML Experiments_________ BASIC

In [ ]:
train_df.columns

In [ ]:
train_df.info()

In [ ]:


# ==== WPROWADŹ SWOJE KOLUMNY ====
categorical_cols = [ 'job', 'marital','education','default','housing','loan','contact','month','poutcome']#,  'no_count', 'unknown_count' ]  # 'loan_plus_housing', <- wpisz swoje kolumny kategoryczne
numerical_cols = ['age','balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'balance_log', 'duration_log', 'day_of_year_sin', 'day_of_year_cos']#'day_of_year_sin', 'day_of_year_cos' ]#, 'previous' ]#'pdays']#, 'previous']    # <- wpisz swoje kolumny numeryczne
target_col = 'y'


In [ ]:
# ==== WCZYTAJ DANE ====
# train_df i test_df muszą być wcześniej wczytane

df = train_df.copy()
X = df[categorical_cols + numerical_cols].copy()
y = df[target_col].copy()

# ==== ENKODOWANIE DANYCH KATEGORYCZNYCH ==== Dla CATBOOSTA MOŻNA POMINĄ TYLKO PODAĆ LISTĘ
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# for col in ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "poutcome"]:
#     X[col] = X[col].astype("category")

# ==== K-FOLD USTAWIENIA ====
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# ==== FUNKCJA TRENOWANIA MODELU ====
def train_model(X, y, params):
    scores = []
    importances = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

        model = XGBClassifier(
            **params,
            eval_metric='auc',
            #enable_categorical=True,
            use_label_encoder=False,
            n_jobs=-1,
         #   scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum()
        )
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)
        scores.append(auc)
        importances.append(model.feature_importances_)

        print(f"Fold {fold+1} ROC-AUC: {auc:.5f}")

    print(f"\nŚredni ROC-AUC: {np.mean(scores):.5f}")
    avg_importances = np.mean(importances, axis=0)

    importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': avg_importances
    }).sort_values(by='importance', ascending=False)

    print("\nFeature Importances:\n", importance_df)

    # Można też narysować
    importance_df.plot.bar(x='feature', y='importance', legend=False,  figsize=(10,5))
    plt.title("Średnia ważność cech (feature importance)")
    plt.show()

### Optuna configuration

In [ ]:
def optuna_search(n_trials=30):
    def objective(trial):
        params = {
            'max_depth': trial.suggest_int('max_depth', 5, 7),#('max_depth', 7, 15),#3-11
            'learning_rate': trial.suggest_float('learning_rate', 0.046, 0.058),
            'n_estimators': trial.suggest_int('n_estimators', 3000, 7000),#('n_estimators', 200, 1200),#200-600
            'subsample': trial.suggest_float('subsample', 0.91, 0.95),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.66, 0.76),
            'gamma': trial.suggest_float('gamma', 0.1, 0.5),
            'reg_alpha': trial.suggest_float('reg_alpha', 2.5,3.5),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.5, 3.8),
# Additional parameters for optuna serch
            'min_child_weight': trial.suggest_int('min_child_weight', 9, 13),#('min_child_weight', 1, 28),
            'max_bin': trial.suggest_int('max_bin', 3800, 5000),#('max_bin', 256, 1024),
            'grow_policy': 'lossguide' #trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        }

        scores = []
        for train_idx, val_idx in kf.split(X, y):
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

            model = XGBClassifier(
                **params,
                eval_metric='auc',
                use_label_encoder=False,
                n_jobs=-1,
             #   scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum()
            )

            model.fit(X_train, y_train)
            y_pred = model.predict_proba(X_val)[:, 1]
            auc = roc_auc_score(y_val, y_pred)
            scores.append(auc)

        return np.mean(scores)


    study = optuna.create_study(direction='maximize')
    #Dodanie najlepszych param z przeszłości
    best_params_prev ={'max_depth': 6, 'learning_rate': 0.05315581672068095, 'n_estimators': 3947, 'subsample': 0.9341116539297873, 'colsample_bytree': 0.7121661138974171, 'gamma': 0.3250659086152249, 'reg_alpha': 3.110520372569841, 'reg_lambda': 2.5699454711304743, 'min_child_weight': 11, 'max_bin': 4074, 'grow_policy': 'lossguide'}  
    study.enqueue_trial(best_params_prev)
    study.optimize(objective, n_trials=n_trials)

    # Przekazanie stałych parametró (nie są zapisywane przez optunę do best_params)
    best_params = study.best_params.copy()
    best_params.update({'grow_policy': 'lossguide'})

    print("\nNajlepsze parametry Optuna:\n", best_params)
    print(f"Najlepszy ROC-AUC: {study.best_value:.5f}")

    # Trenowanie najlepszego modelu na koniec
    train_model(X, y, best_params)

In [ ]:
optuna_search(n_trials=50)

# XGB With early stoping and OPTUNA 

In [ ]:
# ==== WPROWADŹ SWOJE KOLUMNY ====
categorical_cols = [ 'job', 'marital','education','default','housing','loan','contact','month','poutcome']#,  'no_count', 'unknown_count' ]  # 'loan_plus_housing', <- wpisz swoje kolumny kategoryczne
numerical_cols = ['age','balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'balance_log', 'duration_log', 'day_of_year_sin', 'day_of_year_cos']#'day_of_year_sin', 'day_of_year_cos' ]#, 'previous' ]#'pdays']#, 'previous']    # <- wpisz swoje kolumny numeryczne
target_col = 'y'


In [ ]:
# ==== WCZYTAJ DANE ====
# train_df i test_df muszą być wcześniej wczytane

df = train_df.copy()
X = df[categorical_cols + numerical_cols].copy()
y = df[target_col].copy()

# ==== ENKODOWANIE DANYCH KATEGORYCZNYCH ==== Dla CATBOOSTA MOŻNA POMINĄ TYLKO PODAĆ LISTĘ
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# for col in ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "poutcome"]:
#     X[col] = X[col].astype("category")

# ==== K-FOLD USTAWIENIA ====
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# ==== FUNKCJA TRENOWANIA MODELU ====
def train_model(X, y, params):
    scores = []
    importances = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

        model = XGBClassifier(
            **params,
            eval_metric='auc',
            #enable_categorical=True,
            use_label_encoder=False,
            n_jobs=-1,
         #   scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum()
        )
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)
        scores.append(auc)
        importances.append(model.feature_importances_)

        print(f"Fold {fold+1} ROC-AUC: {auc:.5f}")

    print(f"\nŚredni ROC-AUC: {np.mean(scores):.5f}")
    avg_importances = np.mean(importances, axis=0)

    importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': avg_importances
    }).sort_values(by='importance', ascending=False)

    print("\nFeature Importances:\n", importance_df)

    # Można też narysować
    importance_df.plot.bar(x='feature', y='importance', legend=False,  figsize=(10,5))
    plt.title("Średnia ważność cech (feature importance)")
    plt.show()

In [ ]:
def optuna_search(n_trials=30):
    def objective(trial):
        params = {
            'max_depth': trial.suggest_int('max_depth', 5, 7),#('max_depth', 7, 15),#3-11
            'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.05),
            'n_estimators': trial.suggest_int('n_estimators', 3000, 8000),#('n_estimators', 200, 1200),#200-600
            'subsample': trial.suggest_float('subsample', 0.91, 0.98),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.60, 0.76),
            'gamma': trial.suggest_float('gamma', 0.05, 0.5),
            'reg_alpha': trial.suggest_float('reg_alpha', 2.5,3.5),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.3, 3.8),
# Additional parameters for optuna serch
            'min_child_weight': trial.suggest_int('min_child_weight', 7, 12),#('min_child_weight', 1, 28),
            'max_bin': trial.suggest_int('max_bin', 3800, 5000),#('max_bin', 256, 1024),
            'grow_policy': 'lossguide' #trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        }

        scores = []
        for train_idx, val_idx in kf.split(X, y):
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

            model = XGBClassifier(
                **params,
                eval_metric='auc',
                use_label_encoder=False,
                n_jobs=-1,
                early_stopping_rounds=300,
             #   scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum()
            )

            model.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                verbose=1500
            )
            y_pred = model.predict_proba(X_val)[:, 1]
            auc = roc_auc_score(y_val, y_pred)
            scores.append(auc)

        return np.mean(scores)


    study = optuna.create_study(direction='maximize')
    #Dodanie najlepszych param z przeszłości
    best_params_prev ={'max_depth': 7, 'learning_rate': 0.04988569401318322, 'n_estimators': 3718, 'subsample': 0.9665374722246346, 'colsample_bytree': 0.661234616889833, 'gamma': 0.10614619651529983, 'reg_alpha': 3.1062652642283877, 'reg_lambda': 0.7947562715135358, 'min_child_weight': 8, 'max_bin': 4445, 'grow_policy': 'lossguide'}
    study.enqueue_trial(best_params_prev)
    study.optimize(objective, n_trials=n_trials)

    # Przekazanie stałych parametró (nie są zapisywane przez optunę do best_params)
    best_params = study.best_params.copy()
    best_params.update({'grow_policy': 'lossguide'})

    print("\nNajlepsze parametry Optuna:\n", best_params)
    print(f"Najlepszy ROC-AUC: {study.best_value:.5f}")

    # Trenowanie najlepszego modelu na koniec
    #train_model(X, y, best_params)

In [ ]:
optuna_search(n_trials=10)

# _______________________ RESULTS________________________

In [ ]:
# 19
# RAW DATA  With optuna XGBoost
# Public LB Score = 0.96878 bez CV w Submission
# Public LB Score = 0.96924 z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 10, 'learning_rate': 0.088542375425159, 'n_estimators': 353, 'subsample': 0.7933680125492807, 'colsample_bytree': 0.665859932215261, 'gamma': 1.9591038948726576, 'reg_alpha': 4.135658777432998, 'reg_lambda': 3.0378978111339086}
Najlepszy ROC-AUC: 0.96788
Fold 1 ROC-AUC: 0.96859
Fold 2 ROC-AUC: 0.96726
Fold 3 ROC-AUC: 0.96728
Fold 4 ROC-AUC: 0.96853
Fold 5 ROC-AUC: 0.96773

Średni ROC-AUC: 0.96788

Feature Importances:
       feature  importance
12   duration    0.255548
6     contact    0.171966
4     housing    0.160483
8    poutcome    0.070366
7       month    0.061991
14      pdays    0.056551
5        loan    0.044870
10    balance    0.037885
11        day    0.022941
1     marital    0.022816
13   campaign    0.021023
9         age    0.017168
2   education    0.016198
0         job    0.015291
15   previous    0.013801
3     default    0.011103

In [ ]:
# V20
# RAW DATA  With optuna XGBoost bez cechy previous 
# Public LB Score = 0.96914 bez z CV w Submission 
# Public LB Score = 0.96942  z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 9, 'learning_rate': 0.11410300715659989, 'n_estimators': 368, 'subsample': 0.907977121199552, 'colsample_bytree': 0.6014268962223818, 'gamma': 0.7012459904827175, 'reg_alpha': 4.361523501536162, 'reg_lambda': 4.294305690785547}
Najlepszy ROC-AUC: 0.96787
Fold 1 ROC-AUC: 0.96860
Fold 2 ROC-AUC: 0.96714
Fold 3 ROC-AUC: 0.96721
Fold 4 ROC-AUC: 0.96854
Fold 5 ROC-AUC: 0.96785

Średni ROC-AUC: 0.96787

Feature Importances:
       feature  importance
12   duration    0.361053
6     contact    0.166310
4     housing    0.132702
8    poutcome    0.083217
7       month    0.062788
5        loan    0.041027
14      pdays    0.026778
10    balance    0.024013
11        day    0.019122
2   education    0.016243
13   campaign    0.016015
1     marital    0.015528
9         age    0.015053
0         job    0.011771
3     default    0.008379

In [ ]:
# V21
# RAW DATA  With optuna XGBoost bez cechy previous  / z enable_categorical=True (XGBoost sam obsługuje kategorie)
# Public LB Score = 0.96844 bez CV w Submission
# Public LB Score = 0.96891 z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 8, 'learning_rate': 0.07733762308533498, 'n_estimators': 469, 'subsample': 0.9190881144681935, 'colsample_bytree': 0.7461531023839627, 'gamma': 2.06998675412871, 'reg_alpha': 1.3641260353935682, 'reg_lambda': 4.343504135311625}
Najlepszy ROC-AUC: 0.96775
Fold 1 ROC-AUC: 0.96845
Fold 2 ROC-AUC: 0.96711
Fold 3 ROC-AUC: 0.96709
Fold 4 ROC-AUC: 0.96834
Fold 5 ROC-AUC: 0.96777

Średni ROC-AUC: 0.96775

Feature Importances:
       feature  importance
12   duration    0.324780
6     contact    0.173031
4     housing    0.120737
8    poutcome    0.118197
7       month    0.067933
5        loan    0.038657
14      pdays    0.029699
10    balance    0.022180
11        day    0.019063
1     marital    0.017264
0         job    0.016717
13   campaign    0.016680
9         age    0.013868
2   education    0.011933
3     default    0.009262

In [ ]:
# V22
# optuna XGBoost bez cechy previous bez enable_categorical=True, pdays rozdzielone, 'days' jako cat, loan_plus_housing, deep_debt
# Public LB Score = 0.96848 bez CV w Submission
# Public LB Score = 0.96886 z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 9, 'learning_rate': 0.07135259874564061, 'n_estimators': 530, 'subsample': 0.8521429564138164, 'colsample_bytree': 0.7823863939520701, 'gamma': 2.9329829182260148, 'reg_alpha': 3.2901322430190185, 'reg_lambda': 1.268405294791835}
Najlepszy ROC-AUC: 0.96765
Fold 1 ROC-AUC: 0.96834
Fold 2 ROC-AUC: 0.96694
Fold 3 ROC-AUC: 0.96709
Fold 4 ROC-AUC: 0.96824
Fold 5 ROC-AUC: 0.96761

Średni ROC-AUC: 0.96765

Feature Importances:
                 feature  importance
15             duration    0.239886
6               contact    0.185347
10    loan_plus_housing    0.145455
9   no_previous_contact    0.078178
8              poutcome    0.052764
7                 month    0.051926
17        pdays_cleaned    0.048056
4               housing    0.045511
14              balance    0.025864
5                  loan    0.018386
1               marital    0.018161
16             campaign    0.017235
12                  day    0.016298
2             education    0.014973
13                  age    0.013090
0                   job    0.011265
3               default    0.009477
11            deep_debt    0.008127

In [ ]:
# V23
# optuna XGBoost bez cechy previous bez enable_categorical=True, pdays rozdzielone, 'days' jako num, loan_plus_housing
# Public LB Score = 0.96870 bez CV w Submission
# Public LB Score = 0.96895 z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 8, 'learning_rate': 0.12763052427108496, 'n_estimators': 411, 'subsample': 0.953989568332575, 'colsample_bytree': 0.6023438935073028, 'gamma': 0.019491768244553676, 'reg_alpha': 3.961328967768697, 'reg_lambda': 1.131707666650681}

Najlepszy ROC-AUC: 0.96782
Fold 1 ROC-AUC: 0.96848
Fold 2 ROC-AUC: 0.96723
Fold 3 ROC-AUC: 0.96712
Fold 4 ROC-AUC: 0.96847
Fold 5 ROC-AUC: 0.96778

Średni ROC-AUC: 0.96782

Feature Importances:
                 feature  importance
13             duration    0.276787
6               contact    0.190028
10    loan_plus_housing    0.082526
9   no_previous_contact    0.079404
4               housing    0.067073
7                 month    0.058600
15        pdays_cleaned    0.042893
8              poutcome    0.041466
12              balance    0.033900
5                  loan    0.023026
16                  day    0.020086
14             campaign    0.020008
11                  age    0.016162
1               marital    0.014663
2             education    0.013929
3               default    0.009995
0                   job    0.009455

In [ ]:
# V24
# optuna XGBoost bez cechy previous bez enable_categorical=True, pdays rozdzielone, 'days' jako num, loan_plus_housing + col no_count + unknown_count
# Public LB Score = 96831 bez CV w Submission
# Public LB Score = 0.96867 z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 7, 'learning_rate': 0.16686222138398568, 'n_estimators': 359, 'subsample': 0.9687425335082887, 'colsample_bytree': 0.8948829860260885, 'gamma': 0.3972938375538543, 'reg_alpha': 4.575416910972401, 'reg_lambda': 2.201098085025244}

Najlepszy ROC-AUC: 0.96756
Fold 1 ROC-AUC: 0.96836
Fold 2 ROC-AUC: 0.96696
Fold 3 ROC-AUC: 0.96680
Fold 4 ROC-AUC: 0.96824
Fold 5 ROC-AUC: 0.96744

Średni ROC-AUC: 0.96756

Feature Importances:
                 feature  importance
11             no_count    0.273360
15             duration    0.209798
6               contact    0.118712
12        unknown_count    0.075067
9   no_previous_contact    0.067390
7                 month    0.044799
17        pdays_cleaned    0.040017
8              poutcome    0.038668
14              balance    0.016463
2             education    0.015356
10    loan_plus_housing    0.013421
5                  loan    0.013264
18                  day    0.013007
4               housing    0.012936
16             campaign    0.012547
1               marital    0.012498
13                  age    0.009314
0                   job    0.007695
3               default    0.005689

In [ ]:
# V25
# optuna XGBoost bez cechy previous bez enable_categorical=True, pdays rozdzielone, 'days' jako num + col no_count + unknown_count  --bez loan_plus_housing - 
# Public LB Score = 0.96883 bez CV w Submission
# Public LB Score = 0.96900 z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 10, 'learning_rate': 0.05576641173716932, 'n_estimators': 492, 'subsample': 0.7568779660915822, 'colsample_bytree': 0.6384212073745787, 'gamma': 2.0698430928317997, 'reg_alpha': 2.9165075287479447, 'reg_lambda': 2.040935203862332}

Najlepszy ROC-AUC: 0.96792
Fold 1 ROC-AUC: 0.96869
Fold 2 ROC-AUC: 0.96731
Fold 3 ROC-AUC: 0.96734
Fold 4 ROC-AUC: 0.96847
Fold 5 ROC-AUC: 0.96779

Średni ROC-AUC: 0.96792

Feature Importances:
                 feature  importance
14             duration    0.241594
10             no_count    0.142254
6               contact    0.121863
9   no_previous_contact    0.109933
11        unknown_count    0.057236
7                 month    0.056043
8              poutcome    0.044780
16        pdays_cleaned    0.040740
4               housing    0.033472
5                  loan    0.022579
13              balance    0.022401
17                  day    0.018681
15             campaign    0.018105
1               marital    0.017343
2             education    0.015570
12                  age    0.014035
0                   job    0.011947
3               default    0.011424

In [ ]:
# V26
# RAW DATA  With optuna XGBoost bez cechy previous + pdays rozdzielone
# Public LB Score = ----------- bez z CV w Submission 
# Public LB Score = 0.96911 z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 10, 'learning_rate': 0.0832583908499121, 'n_estimators': 551, 'subsample': 0.8758220883852462, 'colsample_bytree': 0.6810585417529508, 'gamma': 2.389202320659961, 'reg_alpha': 0.9587487104983587, 'reg_lambda': 4.103575772005836}
Najlepszy ROC-AUC: 0.96779
Fold 1 ROC-AUC: 0.96867
Fold 2 ROC-AUC: 0.96720
Fold 3 ROC-AUC: 0.96721
Fold 4 ROC-AUC: 0.96839
Fold 5 ROC-AUC: 0.96750

Średni ROC-AUC: 0.96779

Feature Importances:
                 feature  importance
13             duration    0.243189
4               housing    0.139019
6               contact    0.138376
9   no_previous_contact    0.123810
8              poutcome    0.074177
7                 month    0.060123
5                  loan    0.039687
15        pdays_cleaned    0.038699
11              balance    0.027620
12                  day    0.020040
1               marital    0.019900
14             campaign    0.018159
10                  age    0.015864
2             education    0.015750
0                   job    0.014219
3               default    0.011369

In [ ]:
# V27
# RAW DATA  With optuna XGBoost bez cechy previous  + col no_count
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.96901 z CV w Submission 

Najlepsze parametry Optuna:
 {'max_depth': 11, 'learning_rate': 0.07256654502258869, 'n_estimators': 264, 'subsample': 0.7252245807583018, 'colsample_bytree': 0.7225278113013671, 'gamma': 3.044364512727718, 'reg_alpha': 4.9552240198208874, 'reg_lambda': 3.7933495200093277}
Najlepszy ROC-AUC: 0.96786
Fold 1 ROC-AUC: 0.96868
Fold 2 ROC-AUC: 0.96723
Fold 3 ROC-AUC: 0.96730
Fold 4 ROC-AUC: 0.96846
Fold 5 ROC-AUC: 0.96764

Średni ROC-AUC: 0.96786

Feature Importances:
       feature  importance
13   duration    0.261933
6     contact    0.184296
9    no_count    0.156206
4     housing    0.073811
8    poutcome    0.064446
7       month    0.057004
15      pdays    0.043390
11    balance    0.026560
5        loan    0.025262
12        day    0.018575
1     marital    0.018476
14   campaign    0.017962
2   education    0.014880
10        age    0.014734
0         job    0.012134
3     default    0.010333

In [ ]:
# V28
# RAW DATA  With optuna XGBoost bez cechy previous   + col unknown_count
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.96913 z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 11, 'learning_rate': 0.05314481725365295, 'n_estimators': 485, 'subsample': 0.6616608475014232, 'colsample_bytree': 0.6827154636682595, 'gamma': 1.99699754127895, 'reg_alpha': 3.8063665621075478, 'reg_lambda': 1.325699284371194}
Najlepszy ROC-AUC: 0.96787
Fold 1 ROC-AUC: 0.96866
Fold 2 ROC-AUC: 0.96720
Fold 3 ROC-AUC: 0.96733
Fold 4 ROC-AUC: 0.96837
Fold 5 ROC-AUC: 0.96778

Średni ROC-AUC: 0.96787

Feature Importances:
           feature  importance
13       duration    0.246059
4         housing    0.147026
6         contact    0.120959
9   unknown_count    0.095596
8        poutcome    0.083078
7           month    0.063654
5            loan    0.042798
15          pdays    0.040915
11        balance    0.030914
12            day    0.022072
1         marital    0.021628
14       campaign    0.020466
2       education    0.018847
10            age    0.018099
0             job    0.015135
3         default    0.012754

# __________________ NOWE EXPERYMENTY _____________________

In [ ]:
## Sprawdź job_edu matrix
# RAW DATA  With optuna XGBoost bez cechy previous   + col job_edu
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score =  z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 9, 'learning_rate': 0.08212321514497181, 'n_estimators': 523, 'subsample': 0.7525320809093057, 'colsample_bytree': 0.6765190265893073, 'gamma': 3.0377809209553503, 'reg_alpha': 3.625755897052336, 'reg_lambda': 0.18705008615849394}
Najlepszy ROC-AUC: 0.96773
Fold 1 ROC-AUC: 0.96851
Fold 2 ROC-AUC: 0.96708
Fold 3 ROC-AUC: 0.96716
Fold 4 ROC-AUC: 0.96832
Fold 5 ROC-AUC: 0.96758

Średni ROC-AUC: 0.96773

Feature Importances:
               feature  importance
13           duration    0.260276
4             housing    0.164296
6             contact    0.159799
8            poutcome    0.090988
7               month    0.066868
5                loan    0.045981
15              pdays    0.039986
11            balance    0.030999
12                day    0.022041
1             marital    0.021875
14           campaign    0.020257
10                age    0.017447
2           education    0.016907
0                 job    0.015690
9   job_and_education    0.013969
3             default    0.012622

In [ ]:
# V30
# bez weight balanced
# RAW DATA  With optuna XGBoost bez cechy previous, no XGB weight balanced
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.96933 z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 11, 'learning_rate': 0.11206031547278855, 'n_estimators': 501, 'subsample': 0.6131323770348673, 'colsample_bytree': 0.6094568889243491, 'gamma': 2.043078715855179, 'reg_alpha': 2.0852138450654643, 'reg_lambda': 4.796903414902333}
Najlepszy ROC-AUC: 0.96812
Fold 1 ROC-AUC: 0.96889
Fold 2 ROC-AUC: 0.96754
Fold 3 ROC-AUC: 0.96766
Fold 4 ROC-AUC: 0.96853
Fold 5 ROC-AUC: 0.96799

Średni ROC-AUC: 0.96812

Feature Importances:
       feature  importance
12   duration    0.222174
4     housing    0.181200
6     contact    0.177984
8    poutcome    0.135613
5        loan    0.054855
7       month    0.054545
14      pdays    0.029267
10    balance    0.021182
13   campaign    0.021124
11        day    0.021034
2   education    0.020611
1     marital    0.019631
9         age    0.017880
0         job    0.013251
3     default    0.009648

In [ ]:
# V31
# bez weight balanced
# RAW DATA  With optuna XGBoost z previous, no XGB weight balanced
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.96955 z CV w Submission 

Najlepsze parametry Optuna:
 {'max_depth': 9, 'learning_rate': 0.12075088743052882, 'n_estimators': 492, 'subsample': 0.8707220500188539, 'colsample_bytree': 0.6688353258198634, 'gamma': 0.7893885646435537, 'reg_alpha': 0.7438751764100405, 'reg_lambda': 3.4902318090320925}
Najlepszy ROC-AUC: 0.96819
Fold 1 ROC-AUC: 0.96885
Fold 2 ROC-AUC: 0.96757
Fold 3 ROC-AUC: 0.96767
Fold 4 ROC-AUC: 0.96879
Fold 5 ROC-AUC: 0.96806

Średni ROC-AUC: 0.96819

Feature Importances:
       feature  importance
12   duration    0.218552
4     housing    0.176054
6     contact    0.158084
8    poutcome    0.082320
14      pdays    0.065794
7       month    0.064837
5        loan    0.055597
10    balance    0.032892
11        day    0.023744
13   campaign    0.022558
1     marital    0.021204
9         age    0.019373
2   education    0.019224
0         job    0.014700
3     default    0.012958
15   previous    0.012108

In [ ]:
# V32
## RAW DATA  With optuna XGBoost 10 folds z previous bez scale_pos_weight
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.96944 z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 9, 'learning_rate': 0.06227423103688831, 'n_estimators': 556, 'subsample': 0.9298469326500857, 'colsample_bytree': 0.8895510541144902, 'gamma': 0.48209725496163736, 'reg_alpha': 3.796950975692023, 'reg_lambda': 1.6612550809713749}
Najlepszy ROC-AUC: 0.96849
Fold 1 ROC-AUC: 0.96991
Fold 2 ROC-AUC: 0.96855
Fold 3 ROC-AUC: 0.96764
Fold 4 ROC-AUC: 0.96827
Fold 5 ROC-AUC: 0.96771
Fold 6 ROC-AUC: 0.96819
Fold 7 ROC-AUC: 0.96925
Fold 8 ROC-AUC: 0.96863
Fold 9 ROC-AUC: 0.96882
Fold 10 ROC-AUC: 0.96793

Średni ROC-AUC: 0.96849

Feature Importances:
       feature  importance
12   duration    0.227937
4     housing    0.207634
6     contact    0.201319
8    poutcome    0.078979
7       month    0.057670
14      pdays    0.055200
5        loan    0.045558
10    balance    0.021865
13   campaign    0.019530
11        day    0.017168
1     marital    0.015472
2   education    0.013478
9         age    0.012875
0         job    0.010124
15   previous    0.007938
3     default    0.007251

In [ ]:
# V33
## RAW DATA  With optuna XGBoost 10 folds no previuous z scale_pos_weight
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.96903 z CV w Submission 

Najlepsze parametry Optuna:
 {'max_depth': 11, 'learning_rate': 0.04545133522176488, 'n_estimators': 481, 'subsample': 0.7684178974094136, 'colsample_bytree': 0.7986253830153623, 'gamma': 4.481651671654932, 'reg_alpha': 3.3989357087069094, 'reg_lambda': 0.557383341841934}
Najlepszy ROC-AUC: 0.96801
Fold 1 ROC-AUC: 0.96931
Fold 2 ROC-AUC: 0.96811
Fold 3 ROC-AUC: 0.96702
Fold 4 ROC-AUC: 0.96789
Fold 5 ROC-AUC: 0.96726
Fold 6 ROC-AUC: 0.96758
Fold 7 ROC-AUC: 0.96883
Fold 8 ROC-AUC: 0.96835
Fold 9 ROC-AUC: 0.96813
Fold 10 ROC-AUC: 0.96766

Średni ROC-AUC: 0.96801

Feature Importances:
       feature  importance
12   duration    0.272672
6     contact    0.210457
4     housing    0.171070
8    poutcome    0.075329
7       month    0.057601
5        loan    0.040058
14      pdays    0.039956
10    balance    0.022206
11        day    0.018679
13   campaign    0.018674
1     marital    0.018211
2   education    0.016141
9         age    0.015146
0         job    0.013055
3     default    0.010745

In [ ]:
# V34
#'min_child_weight': trial.suggest_int('min_child_weight', 1, 28),
#'max_bin': trial.suggest_int('max_bin', 256, 1024),
#'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),

## RAW DATA  With optuna XGBoost 5 folds, no previuous z scale_pos_weight, n_trial=40
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97147 z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 11, 'learning_rate': 0.11017938521753814, 'n_estimators': 447, 'subsample': 0.7200594556282529, 'colsample_bytree': 0.7298687513204829, 'gamma': 3.7349463296256253, 'reg_alpha': 2.2870628091572853, 'reg_lambda': 0.3821213759830635, 'min_child_weight': 22, 'max_bin': 965, 'grow_policy': 'lossguide'}
Najlepszy ROC-AUC: 0.97010
Fold 1 ROC-AUC: 0.97083
Fold 2 ROC-AUC: 0.96943
Fold 3 ROC-AUC: 0.96969
Fold 4 ROC-AUC: 0.97058
Fold 5 ROC-AUC: 0.96998

Średni ROC-AUC: 0.97010

Feature Importances:
       feature  importance
12   duration    0.223254
6     contact    0.207627
4     housing    0.139693
8    poutcome    0.109910
7       month    0.061940
5        loan    0.057761
14      pdays    0.044007
10    balance    0.023326
11        day    0.021511
1     marital    0.021244
13   campaign    0.020844
2   education    0.020157
9         age    0.017688
3     default    0.015842
0         job    0.015194


In [ ]:
# V35
#'min_child_weight': trial.suggest_int('min_child_weight', 1, 28),
#'max_bin': trial.suggest_int('max_bin', 256, 1024),
#'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),

## RAW DATA  With optuna XGBoost 5 folds, z previuous, no scale_pos_weight, n_trial=40
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97150 z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 10, 'learning_rate': 0.11419656400547268, 'n_estimators': 382, 'subsample': 0.8761249869278733, 'colsample_bytree': 0.9302155455418724, 'gamma': 0.922723932134517, 'reg_alpha': 1.0912230662828875, 'reg_lambda': 2.6136637861620295, 'min_child_weight': 23, 'max_bin': 923, 'grow_policy': 'lossguide'}
Najlepszy ROC-AUC: 0.97018
Fold 1 ROC-AUC: 0.97100
Fold 2 ROC-AUC: 0.96970
Fold 3 ROC-AUC: 0.96957
Fold 4 ROC-AUC: 0.97062
Fold 5 ROC-AUC: 0.97004

Średni ROC-AUC: 0.97018

Feature Importances:
       feature  importance
6     contact    0.217533
4     housing    0.180848
12   duration    0.163919
8    poutcome    0.099551
5        loan    0.072457
14      pdays    0.068149
7       month    0.055700
13   campaign    0.021170
10    balance    0.019574
11        day    0.018934
1     marital    0.018691
2   education    0.016664
9         age    0.014356
0         job    0.011392
3     default    0.011143
15   previous    0.009918

In [ ]:
# V36
#'min_child_weight': trial.suggest_int('min_child_weight', 1, 28),
#'max_bin': trial.suggest_int('max_bin', 256, 1024),
#'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),

## RAW DATA  With optuna XGBoost 5 folds, z previuous, z scale_pos_weight, n_trial=40
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97142 z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 10, 'learning_rate': 0.08502933965618005, 'n_estimators': 492, 'subsample': 0.8475568692723069, 'colsample_bytree': 0.7937125702771929, 'gamma': 2.090595137285895, 'reg_alpha': 2.2649299117163197, 'reg_lambda': 4.582521266327681, 'min_child_weight': 18, 'max_bin': 850, 'grow_policy': 'lossguide'}
Najlepszy ROC-AUC: 0.97005
Fold 1 ROC-AUC: 0.97077
Fold 2 ROC-AUC: 0.96942
Fold 3 ROC-AUC: 0.96942
Fold 4 ROC-AUC: 0.97058
Fold 5 ROC-AUC: 0.97004

Średni ROC-AUC: 0.97005

Feature Importances:
       feature  importance
12   duration    0.209513
6     contact    0.177149
4     housing    0.176229
8    poutcome    0.087599
14      pdays    0.063545
7       month    0.059903
5        loan    0.055038
10    balance    0.030265
1     marital    0.023010
13   campaign    0.021739
11        day    0.020944
2   education    0.016545
3     default    0.015434
9         age    0.015278
15   previous    0.014785
0         job    0.013024

In [ ]:
# V37
#'max_depth' 7, 15
#'n_estimators': 200, 1200)
#'min_child_weight': trial.suggest_int('min_child_weight', 1, 28),
#'max_bin': trial.suggest_int('max_bin', 256, 1024),
#'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),

## RAW DATA  With optuna XGBoost 5 folds, no previuous z scale_pos_weight, n_trial=150
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97179 z CV w Submission 
with value: 0.9706775741350937 and parameters: {'max_depth': 13, 'learning_rate': 0.051597964533266545, 'n_estimators': 1199, 'subsample': 0.9486843342638015, 'colsample_bytree': 0.7012804860738411, 'gamma': 1.4669905642678631, 'reg_alpha': 3.8750691242340927, 'reg_lambda': 3.5772297879036534, 'min_child_weight': 17, 'max_bin': 1024, 'grow_policy': 'depthwise'}
with value: 0.9705044230332828 and parameters: {'max_depth': 11, 'learning_rate': 0.04259500015997499, 'n_estimators': 865, 'subsample': 0.8394497002983581, 'colsample_bytree': 0.6339824727397483, 'gamma': 3.1012031163583353, 'reg_alpha': 2.7765300977121243, 'reg_lambda': 3.97300406750927, 'min_child_weight': 10, 'max_bin': 1000, 'grow_policy': 'lossguide'}. Best is trial 69 with value: 0.9705044230332828.

Najlepsze parametry Optuna:
 {'max_depth': 13, 'learning_rate': 0.051597964533266545, 'n_estimators': 1199, 'subsample': 0.9486843342638015, 'colsample_bytree': 0.7012804860738411, 'gamma': 1.4669905642678631, 'reg_alpha': 3.8750691242340927, 'reg_lambda': 3.5772297879036534, 'min_child_weight': 17, 'max_bin': 1024, 'grow_policy': 'depthwise'}
Najlepszy ROC-AUC: 0.97068
Fold 1 ROC-AUC: 0.97139
Fold 2 ROC-AUC: 0.97019
Fold 3 ROC-AUC: 0.97006
Fold 4 ROC-AUC: 0.97114
Fold 5 ROC-AUC: 0.97061

Średni ROC-AUC: 0.97068

Feature Importances:
       feature  importance
12   duration    0.229621
6     contact    0.219436
4     housing    0.148120
8    poutcome    0.105272
7       month    0.061781
5        loan    0.058197
14      pdays    0.044451
10    balance    0.019561
11        day    0.018427
1     marital    0.018276
13   campaign    0.018216
2   education    0.016455
3     default    0.015356
9         age    0.014330
0         job    0.012503

In [ ]:
# V38
#'min_child_weight': trial.suggest_int('min_child_weight', 1, 28),
#'max_bin': trial.suggest_int('max_bin', 256, 1024),
#'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),

## RAW DATA  With optuna XGBoost 5 folds, no previuous z scale_pos_weight, n_trial=40, new cols  sin/cos
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97147 z CV w Submission 
Najlepsze parametry Optuna:
 {'max_depth': 10, 'learning_rate': 0.12547508160977527, 'n_estimators': 558, 'subsample': 0.9603467732508381, 'colsample_bytree': 0.6775794698624192, 'gamma': 1.0653180442504713, 'reg_alpha': 3.49579539155107, 'reg_lambda': 3.985155368089634, 'min_child_weight': 12, 'max_bin': 787, 'grow_policy': 'lossguide'}
Najlepszy ROC-AUC: 0.96976
Fold 1 ROC-AUC: 0.97053
Fold 2 ROC-AUC: 0.96929
Fold 3 ROC-AUC: 0.96914
Fold 4 ROC-AUC: 0.97021
Fold 5 ROC-AUC: 0.96964

Średni ROC-AUC: 0.96976

Feature Importances:
             feature  importance
12         duration    0.226128
6           contact    0.220253
4           housing    0.124076
8          poutcome    0.091663
5              loan    0.050570
14            pdays    0.050106
16  day_of_year_cos    0.038176
15  day_of_year_sin    0.035609
7             month    0.031246
10          balance    0.025604
3           default    0.018384
11              day    0.017046
13         campaign    0.016098
1           marital    0.015935
2         education    0.014467
9               age    0.012868
0               job    0.011770


In [ ]:
# V40
#'max_depth' 7, 15
#'n_estimators': 200, 1200)
#'min_child_weight': trial.suggest_int('min_child_weight', 1, 28),
#'max_bin': trial.suggest_int('max_bin', 256, 1024),
#'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),

## RAW DATA  With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight, n_trial=150
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97186  z CV w Submission 

Najlepsze parametry Optuna:
 {'max_depth': 14, 'learning_rate': 0.02701453181796079, 'n_estimators': 1095, 'subsample': 0.8023378542955938, 'colsample_bytree': 0.6104508809024989, 'gamma': 0.45421103247113814, 'reg_alpha': 1.1316365943111237, 'reg_lambda': 1.6012566769586427, 'min_child_weight': 18, 'max_bin': 989}
Najlepszy ROC-AUC: 0.97104
Fold 1 ROC-AUC: 0.97187
Fold 2 ROC-AUC: 0.97052
Fold 3 ROC-AUC: 0.97061
Fold 4 ROC-AUC: 0.97139
Fold 5 ROC-AUC: 0.97081

Średni ROC-AUC: 0.97104

Feature Importances:
       feature  importance
6     contact    0.198984
12   duration    0.163953
8    poutcome    0.148052
4     housing    0.123673
5        loan    0.066679
7       month    0.065384
14      pdays    0.063445
15   previous    0.023446
10    balance    0.022897
1     marital    0.021821
11        day    0.020702
13   campaign    0.019559
2   education    0.016312
9         age    0.016251
3     default    0.015141
0         job    0.013703

In [ ]:
#V59
beeest_param ={'max_depth': 7, 'learning_rate': 0.04988569401318322, 'n_estimators': 3718, 'subsample': 0.9665374722246346, 'colsample_bytree': 0.661234616889833, 'gamma': 0.10614619651529983, 'reg_alpha': 3.1062652642283877, 'reg_lambda': 0.7947562715135358, 'min_child_weight': 8, 'max_bin': 4445, 'grow_policy': 'lossguide'}

In [ ]:
train_model(X, y, beeest_param)

In [ ]:
# V41
## RAW DATA  With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight,
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97248  z CV w Submission 

# Trial 1 finished with value: 0.9715865904724591 and parameters: {'max_depth': 14, 'learning_rate': 0.01624073742578193, 'n_estimators': 1632, 'subsample': 0.899429787551878, 'colsample_bytree': 0.6185268489803915, 'gamma': 0.36362513980290667, 'reg_alpha': 0.744367728380721, 'reg_lambda': 0.4628340619185789, 'min_child_weight': 9, 'max_bin': 1401, 'grow_policy': 'lossguide'}

Fold 1 ROC-AUC: 0.97236
Fold 2 ROC-AUC: 0.97114
Fold 3 ROC-AUC: 0.97101
Fold 4 ROC-AUC: 0.97196
Fold 5 ROC-AUC: 0.97151

Średni ROC-AUC: 0.97160

Feature Importances:
       feature  importance
6     contact    0.198861
12   duration    0.167826
4     housing    0.138277
8    poutcome    0.137458
7       month    0.066364
5        loan    0.066318
14      pdays    0.055454
15   previous    0.023862
10    balance    0.022934
11        day    0.020478
1     marital    0.020246
13   campaign    0.019228
3     default    0.016890
9         age    0.015989
2   education    0.015786
0         job    0.014029

In [ ]:
# V42
## RAW DATA  With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight,
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97316 z CV w Submission 

#Trial 11 finished with value: 0.9718884977490984 and parameters: {'max_depth': 19, 'learning_rate': 0.08091498981682245, 'n_estimators': 300, 'subsample': 0.8533765487098182, 'colsample_bytree': 0.6026759464047292, 'gamma': 0.0043501519621497176, 'reg_alpha': 3.4621137407155906, 'reg_lambda': 0.07698105946770128, 'min_child_weight': 1, 'max_bin': 2019, 'grow_policy': 'lossguide'}

Fold 1 ROC-AUC: 0.97269
Fold 2 ROC-AUC: 0.97152
Fold 3 ROC-AUC: 0.97113
Fold 4 ROC-AUC: 0.97229
Fold 5 ROC-AUC: 0.97184

Średni ROC-AUC: 0.97189

Feature Importances:
       feature  importance
12   duration    0.192274
4     housing    0.163926
6     contact    0.159542
7       month    0.084574
8    poutcome    0.080662
14      pdays    0.078938
5        loan    0.046858
10    balance    0.032551
11        day    0.028901
13   campaign    0.024628
1     marital    0.022266
9         age    0.020955
2   education    0.019305
0         job    0.017373
15   previous    0.016016
3     default    0.011230


In [ ]:
# V43
## RAW DATA  With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight,
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0,97282 z CV w Submission 

#vTrial 15 finished with value: 0.9717732494056683 and parameters: {'max_depth': 17, 'learning_rate': 0.06519980970054881, 'n_estimators': 680, 'subsample': 0.8736025882716271, 'colsample_bytree': 0.875664321925202, 'gamma': 1.174374070219337, 'reg_alpha': 4.517748889976481, 'reg_lambda': 0.8719349977849177, 'min_child_weight': 5, 'max_bin': 1770, 'grow_policy': 'lossguide'}

Fold 1 ROC-AUC: 0.97257
Fold 2 ROC-AUC: 0.97149
Fold 3 ROC-AUC: 0.97134
Fold 4 ROC-AUC: 0.97222
Fold 5 ROC-AUC: 0.97177

Średni ROC-AUC: 0.97188

Feature Importances:
       feature  importance
4     housing    0.252498
6     contact    0.221672
12   duration    0.145096
8    poutcome    0.092546
14      pdays    0.063781
7       month    0.052611
5        loan    0.043339
13   campaign    0.020093
10    balance    0.018717
11        day    0.016958
1     marital    0.016530
2   education    0.014053
9         age    0.012917
0         job    0.010208
3     default    0.009907
15   previous    0.009073

In [ ]:
# V44
## RAW DATA  With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight,
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0,97299 z CV w Submission 

#Trial 13 finished with value: 0.972124477830939 and parameters: {'max_depth': 18, 'learning_rate': 0.0708831450623464, 'n_estimators': 553, 'subsample': 0.8546933847917322, 'colsample_bytree': 0.7477903378445798, 'gamma': 1.196030349250854, 'reg_alpha': 3.387163897627949, 'reg_lambda': 0.8826864133067371, 'min_child_weight': 1, 'max_bin': 2022, 'grow_policy': 'lossguide'}
    
Fold 1 ROC-AUC: 0.97273
Fold 2 ROC-AUC: 0.97155
Fold 3 ROC-AUC: 0.97147
Fold 4 ROC-AUC: 0.97244
Fold 5 ROC-AUC: 0.97187

Średni ROC-AUC: 0.97201

Feature Importances:
       feature  importance
4     housing    0.243209
6     contact    0.185113
12   duration    0.142803
8    poutcome    0.099292
14      pdays    0.066120
7       month    0.061654
5        loan    0.051452
10    balance    0.023324
13   campaign    0.020940
11        day    0.020490
1     marital    0.018648
9         age    0.016422
2   education    0.015506
0         job    0.012743
15   previous    0.011924
3     default    0.010359

In [ ]:
# V45
## RAW DATA  With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight,
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97344 z CV w Submission 

Trial 19 finished with value: 0.972428270544175 and parameters: {'max_depth': 16, 'learning_rate': 0.0444106455843034, 'n_estimators': 2746, 'subsample': 0.8385441637974808, 'colsample_bytree': 0.8660799923129452, 'gamma': 0.6988818955694143, 'reg_alpha': 4.026337640164294, 'reg_lambda': 4.0256180856994215, 'min_child_weight': 3, 'max_bin': 1904, 'grow_policy': 'lossguide'}

Fold 1 ROC-AUC: 0.97322
Fold 2 ROC-AUC: 0.97202
Fold 3 ROC-AUC: 0.97177
Fold 4 ROC-AUC: 0.97278
Fold 5 ROC-AUC: 0.97241

Średni ROC-AUC: 0.97244

Feature Importances:
       feature  importance
4     housing    0.240487
6     contact    0.187580
12   duration    0.145691
8    poutcome    0.096370
14      pdays    0.064628
7       month    0.063831
5        loan    0.043229
13   campaign    0.022724
10    balance    0.022455
11        day    0.020449
1     marital    0.019210
2   education    0.017432
9         age    0.016351
0         job    0.013964
3     default    0.013399
15   previous    0.012203


In [ ]:
# V47
## RAW DATA  With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight,
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97356 z CV w Submission 

Trial 60 finished with value: 0.9724772622087506 and parameters: {'max_depth': 8, 'learning_rate': 0.05003659706480022, 'n_estimators': 3599, 'subsample': 0.8429878804233633, 'colsample_bytree': 0.811904951081627, 'gamma': 0.4013365830244742, 'reg_alpha': 3.6000725691461786, 'reg_lambda': 0.23781092620605504, 'min_child_weight': 13, 'max_bin': 2442, 'grow_policy': 'lossguide'}

Fold 1 ROC-AUC: 0.97317
Fold 2 ROC-AUC: 0.97208
Fold 3 ROC-AUC: 0.97193
Fold 4 ROC-AUC: 0.97281
Fold 5 ROC-AUC: 0.97240

Średni ROC-AUC: 0.97248

Feature Importances:
       feature  importance
6     contact    0.186063
4     housing    0.182291
8    poutcome    0.133523
12   duration    0.131369
5        loan    0.075332
14      pdays    0.064848
7       month    0.062476
10    balance    0.022873
13   campaign    0.022584
1     marital    0.020933
11        day    0.020079
2   education    0.018259
9         age    0.016676
15   previous    0.015025
0         job    0.013968
3     default    0.013700


In [ ]:
# V48
## RAW DATA  With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight,
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97353 z CV w Submission 

Trial 63 finished with value: 0.9726055439804251 and parameters: {'max_depth': 8, 'learning_rate': 0.030916714630792767, 'n_estimators': 3960, 'subsample': 0.8688930835089947, 'colsample_bytree': 0.8464859741187998, 'gamma': 0.03762546754890961, 'reg_alpha': 3.6580944723435667, 'reg_lambda': 0.00489088726961423, 'min_child_weight': 13, 'max_bin': 2127, 'grow_policy': 'lossguide'}

Fold 1 ROC-AUC: 0.97335
Fold 2 ROC-AUC: 0.97224
Fold 3 ROC-AUC: 0.97198
Fold 4 ROC-AUC: 0.97298
Fold 5 ROC-AUC: 0.97247

Średni ROC-AUC: 0.97261

Feature Importances:
       feature  importance
6     contact    0.214378
4     housing    0.170146
12   duration    0.138952
8    poutcome    0.128084
5        loan    0.077582
14      pdays    0.063418
7       month    0.061083
13   campaign    0.021382
10    balance    0.020061
1     marital    0.018517
11        day    0.017632
2   education    0.016848
9         age    0.014558
15   previous    0.012649
3     default    0.012355
0         job    0.012353




In [ ]:
# V49
## RAW DATA  With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight, ntrials120
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97362 z CV w Submission 
Trial 25 finished with value: 0.972900950622361 and parameters: {'max_depth': 8, 'learning_rate': 0.014826977628385373, 'n_estimators': 5465, 'subsample': 0.8043315478432289, 'colsample_bytree': 0.645060082872195, 'gamma': 0.20124387942031607, 'reg_alpha': 2.080967542693322, 'reg_lambda': 1.9436897009649072, 'min_child_weight': 5, 'max_bin': 2692, 'grow_policy': 'lossguide'}

Fold 1 ROC-AUC: 0.97357
Fold 2 ROC-AUC: 0.97253
Fold 3 ROC-AUC: 0.97236
Fold 4 ROC-AUC: 0.97323
Fold 5 ROC-AUC: 0.97281

Średni ROC-AUC: 0.97290

Feature Importances:
       feature  importance
12   duration    0.179430
6     contact    0.171820
4     housing    0.168091
8    poutcome    0.127128
5        loan    0.066443
7       month    0.064981
14      pdays    0.054307
10    balance    0.024881
13   campaign    0.021180
11        day    0.020193
1     marital    0.019454
15   previous    0.019347
2   education    0.017835
9         age    0.017737
0         job    0.013877
3     default    0.013296


In [ ]:
# V50
## RAW DATA  With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight, ntrials120
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97381 z CV w Submission 
Trial 15 finished with value: 0.9729172041836366 and parameters: {'max_depth': 9, 'learning_rate': 0.027882762514711706, 'n_estimators': 4807, 'subsample': 0.8738983056484187, 'colsample_bytree': 0.7432234444382687, 'gamma': 0.6872650492187076, 'reg_alpha': 2.5245822386447596, 'reg_lambda': 0.7655196827452664, 'min_child_weight': 12, 'max_bin': 3023, 'grow_policy': 'lossguide'}

Fold 1 ROC-AUC: 0.97363
Fold 2 ROC-AUC: 0.97249
Fold 3 ROC-AUC: 0.97231
Fold 4 ROC-AUC: 0.97325
Fold 5 ROC-AUC: 0.97284

Średni ROC-AUC: 0.97290

Feature Importances:
       feature  importance
6     contact    0.194690
4     housing    0.165211
8    poutcome    0.144438
12   duration    0.138640
5        loan    0.072588
7       month    0.063244
14      pdays    0.059047
13   campaign    0.021942
10    balance    0.021201
11        day    0.019828
1     marital    0.019265
15   previous    0.018764
2   education    0.017925
9         age    0.016414
0         job    0.013885
3     default    0.012920


In [ ]:
# V51
## RAW DATA  With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight, ntrials120
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97378 z CV w Submission 
Trial 13 finished with value: 0.9729088695690926 and parameters: {'max_depth': 10, 'learning_rate': 0.027455440776136326, 'n_estimators': 5934, 'subsample': 0.8465104721153287, 'colsample_bytree': 0.7026509223067279, 'gamma': 0.7470441137061977, 'reg_alpha': 3.1818845226707695, 'reg_lambda': 1.8736642668155106, 'min_child_weight': 12, 'max_bin': 3197, 'grow_policy': 'lossguide'}

Fold 1 ROC-AUC: 0.97360
Fold 2 ROC-AUC: 0.97244
Fold 3 ROC-AUC: 0.97231
Fold 4 ROC-AUC: 0.97319
Fold 5 ROC-AUC: 0.97285

Średni ROC-AUC: 0.97288

Feature Importances:
       feature  importance
6     contact    0.190789
4     housing    0.173433
8    poutcome    0.143425
12   duration    0.133778
5        loan    0.067936
7       month    0.064902
14      pdays    0.060627
13   campaign    0.022205
10    balance    0.021027
11        day    0.020459
1     marital    0.019814
15   previous    0.018675
2   education    0.018217
9         age    0.016624
0         job    0.014230
3     default    0.013857


In [ ]:
# V52
## RAW DATA  With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight, ntrials120
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97373 z CV w Submission 
Trial 44 finished with value: 0.9728931597466186 and parameters: {'max_depth': 10, 'learning_rate': 0.028355231960286054, 'n_estimators': 4844, 'subsample': 0.9250167487880581, 'colsample_bytree': 0.7381558108905986, 'gamma': 0.5304502375744603, 'reg_alpha': 2.8679569811284966, 'reg_lambda': 2.78965614497557, 'min_child_weight': 11, 'max_bin': 2413, 'grow_policy': 'lossguide'}

Fold 1 ROC-AUC: 0.97360
Fold 2 ROC-AUC: 0.97249
Fold 3 ROC-AUC: 0.97229
Fold 4 ROC-AUC: 0.97322
Fold 5 ROC-AUC: 0.97278

Średni ROC-AUC: 0.97288

Feature Importances:
       feature  importance
6     contact    0.193349
4     housing    0.171055
12   duration    0.144127
8    poutcome    0.141395
5        loan    0.069780
7       month    0.063300
14      pdays    0.060723
13   campaign    0.021110
10    balance    0.020970
11        day    0.019485
1     marital    0.018654
2   education    0.017032
15   previous    0.016890
9         age    0.015648
3     default    0.013365
0         job    0.013116

In [ ]:
# V53
## RAW DATA + sin/cos +bal/dur-log With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight, ntrials60
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97383 z CV w Submission 

Trial 18 finished with value: 0.9728263310619216 and parameters: {'max_depth': 20, 'learning_rate': 0.012775441668234806, 'n_estimators': 6911, 'subsample': 0.985977555279653, 'colsample_bytree': 0.6695853179005565, 'gamma': 0.14624951968603828, 'reg_alpha': 0.7772866902374195, 'reg_lambda': 4.016941662581965, 'min_child_weight': 25, 'max_bin': 2987, 'grow_policy': 'lossguide'}

Fold 1 ROC-AUC: 0.97358
Fold 2 ROC-AUC: 0.97247
Fold 3 ROC-AUC: 0.97231
Fold 4 ROC-AUC: 0.97300
Fold 5 ROC-AUC: 0.97277

Średni ROC-AUC: 0.97283

Feature Importances:
             feature  importance
6           contact    0.209585
4           housing    0.138770
8          poutcome    0.133493
17     duration_log    0.090537
12         duration    0.078234
5              loan    0.071323
15         previous    0.044548
14            pdays    0.038886
7             month    0.030434
19  day_of_year_cos    0.029462
18  day_of_year_sin    0.027121
1           marital    0.015059
13         campaign    0.014664
2         education    0.013127
11              day    0.012073
10          balance    0.011614
16      balance_log    0.011472
9               age    0.010821
0               job    0.010569
3           default    0.008208


In [ ]:
# V54
## RAW DATA + sin/cos +bal/dur-log With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight, ntrials60
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97393 z CV w Submission 
Trial 11 finished with value: 0.9729171214867007 and parameters: {'max_depth': 5, 'learning_rate': 0.051544348346341265, 'n_estimators': 5986, 'subsample': 0.9406563987565996, 'colsample_bytree': 0.7032285244144026, 'gamma': 0.014005559892251412, 'reg_alpha': 4.47865750447716, 'reg_lambda': 2.5024030571202474, 'min_child_weight': 10, 'max_bin': 2801, 'grow_policy': 'lossguide'}
Fold 1 ROC-AUC: 0.97370
Fold 2 ROC-AUC: 0.97264
Fold 3 ROC-AUC: 0.97246
Fold 4 ROC-AUC: 0.97314
Fold 5 ROC-AUC: 0.97287

Średni ROC-AUC: 0.97296

Feature Importances:
             feature  importance
6           contact    0.161472
4           housing    0.159185
8          poutcome    0.127285
17     duration_log    0.116859
12         duration    0.100009
5              loan    0.062911
15         previous    0.041243
19  day_of_year_cos    0.035196
18  day_of_year_sin    0.033469
14            pdays    0.033364
7             month    0.026708
13         campaign    0.015524
2         education    0.014928
1           marital    0.012927
16      balance_log    0.011712
10          balance    0.010917
11              day    0.010455
9               age    0.010252
0               job    0.009607
3           default    0.005978

In [ ]:
# V55
## RAW DATA + sin/cos +bal/dur-log With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight, ntrials60
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97395 z CV w Submission 
Trial 51 finished with value: 0.9729235763189358 and parameters: {'max_depth': 7, 'learning_rate': 0.053361175562108204, 'n_estimators': 4017, 'subsample': 0.9306723099635541, 'colsample_bytree': 0.7173136524092636, 'gamma': 0.3817608204886543, 'reg_alpha': 3.534578539601053, 'reg_lambda': 0.37926302495130126, 'min_child_weight': 11, 'max_bin': 3155, 'grow_policy': 'lossguide'}

Fold 1 ROC-AUC: 0.97364
Fold 2 ROC-AUC: 0.97262
Fold 3 ROC-AUC: 0.97238
Fold 4 ROC-AUC: 0.97321
Fold 5 ROC-AUC: 0.97284

Średni ROC-AUC: 0.97294

Feature Importances:
             feature  importance
4           housing    0.166456
6           contact    0.140955
8          poutcome    0.121657
17     duration_log    0.115246
12         duration    0.101595
5              loan    0.062655
15         previous    0.042829
14            pdays    0.038633
19  day_of_year_cos    0.034681
18  day_of_year_sin    0.032759
7             month    0.032162
13         campaign    0.015969
2         education    0.015005
1           marital    0.014020
16      balance_log    0.012849
11              day    0.012253
10          balance    0.011670
9               age    0.010911
0               job    0.009898
3           default    0.007799

In [ ]:
# V56
## RAW DATA + sin/cos +bal/dur-log With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight, ntrials60
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97399 z CV w Submission 
Trial 52 finished with value: 0.9729633428371176 and parameters: {'max_depth': 6, 'learning_rate': 0.05315581672068095, 'n_estimators': 3947, 'subsample': 0.9341116539297873, 'colsample_bytree': 0.7121661138974171, 'gamma': 0.3250659086152249, 'reg_alpha': 3.110520372569841, 'reg_lambda': 2.5699454711304743, 'min_child_weight': 11, 'max_bin': 4074, 'grow_policy': 'lossguide'}

Fold 1 ROC-AUC: 0.97371
Fold 2 ROC-AUC: 0.97277
Fold 3 ROC-AUC: 0.97245
Fold 4 ROC-AUC: 0.97320
Fold 5 ROC-AUC: 0.97279

Średni ROC-AUC: 0.97298

Feature Importances:
             feature  importance
4           housing    0.164956
6           contact    0.149885
8          poutcome    0.121458
17     duration_log    0.115068
12         duration    0.099955
5              loan    0.065144
15         previous    0.042928
19  day_of_year_cos    0.035192
14            pdays    0.034566
18  day_of_year_sin    0.033121
7             month    0.029720
13         campaign    0.016207
2         education    0.015410
1           marital    0.013640
16      balance_log    0.013512
11              day    0.011289
10          balance    0.010680
9               age    0.010557
0               job    0.009887
3           default    0.006827

In [ ]:
# V57
## RAW DATA + sin/cos +bal/dur-log With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight, ntrials60
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97432 z CV w Submission 
Trial 44 finished with value: 0.9731302781471577 and parameters: {'max_depth': 7, 'learning_rate': 0.050010678399715854, 'n_estimators': 4225, 'subsample': 0.9675105389521785, 'colsample_bytree': 0.6642729612066018, 'gamma': 0.06057145608089742, 'reg_alpha': 3.0245216337937118, 'reg_lambda': 0.6834653417762062, 'min_child_weight': 8, 'max_bin': 4403, 'grow_policy': 'lossguide'}
Fold 1 ROC-AUC: 0.97380
Fold 2 ROC-AUC: 0.97293
Fold 3 ROC-AUC: 0.97255
Fold 4 ROC-AUC: 0.97338
Fold 5 ROC-AUC: 0.97307

Średni ROC-AUC: 0.97314

Feature Importances:
             feature  importance
4           housing    0.152681
6           contact    0.139607
17     duration_log    0.130443
8          poutcome    0.123028
12         duration    0.096044
5              loan    0.064063
15         previous    0.043497
14            pdays    0.038821
7             month    0.034475
19  day_of_year_cos    0.034353
18  day_of_year_sin    0.030871
13         campaign    0.016088
2         education    0.014709
1           marital    0.014119
16      balance_log    0.012847
10          balance    0.012293
11              day    0.011895
9               age    0.011109
0               job    0.010338
3           default    0.008718

In [ ]:
# V58
## RAW DATA + sin/cos +bal/dur-log With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight, ntrials60
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97428 z CV w Submission 
Trial 12 finished with value: 0.9731543076169384 and parameters: {'max_depth': 6, 'learning_rate': 0.04616333861262938, 'n_estimators': 5721, 'subsample': 0.9493894816676675, 'colsample_bytree': 0.6658104623618158, 'gamma': 0.21914049298728847, 'reg_alpha': 3.290083253769517, 'reg_lambda': 0.6407462194645503, 'min_child_weight': 9, 'max_bin': 4279, 'grow_policy': 'lossguide'}
Fold 1 ROC-AUC: 0.97384
Fold 2 ROC-AUC: 0.97289
Fold 3 ROC-AUC: 0.97263
Fold 4 ROC-AUC: 0.97339
Fold 5 ROC-AUC: 0.97301

Średni ROC-AUC: 0.97315

Feature Importances:
             feature  importance
4           housing    0.160239
6           contact    0.146110
17     duration_log    0.126703
8          poutcome    0.123444
12         duration    0.094849
5              loan    0.062028
15         previous    0.043875
14            pdays    0.036040
19  day_of_year_cos    0.035144
18  day_of_year_sin    0.032639
7             month    0.030280
13         campaign    0.016021
2         education    0.015850
1           marital    0.013978
10          balance    0.012089
16      balance_log    0.011556
11              day    0.011344
9               age    0.010868
0               job    0.009932
3           default    0.007012

In [ ]:
# V59
## RAW DATA + sin/cos +bal/dur-log With optuna XGBoost 5 folds, z previuous, bez scale_pos_weight, ntrials60
# Public LB Score = -------------- bez z CV w Submission 
# Public LB Score = 0.97427 z CV w Submission 
Trial 45 finished with value: 0.9731549435039618 and parameters: {'max_depth': 7, 'learning_rate': 0.04988569401318322, 'n_estimators': 3718, 'subsample': 0.9665374722246346, 'colsample_bytree': 0.661234616889833, 'gamma': 0.10614619651529983, 'reg_alpha': 3.1062652642283877, 'reg_lambda': 0.7947562715135358, 'min_child_weight': 8, 'max_bin': 4445, 'grow_policy': 'lossguide'}
Fold 1 ROC-AUC: 0.97380
Fold 2 ROC-AUC: 0.97295
Fold 3 ROC-AUC: 0.97260
Fold 4 ROC-AUC: 0.97339
Fold 5 ROC-AUC: 0.97298

Średni ROC-AUC: 0.97314

Feature Importances:
             feature  importance
4           housing    0.158689
6           contact    0.140201
17     duration_log    0.131000
8          poutcome    0.120465
12         duration    0.096493
5              loan    0.062487
15         previous    0.043359
14            pdays    0.038404
19  day_of_year_cos    0.034242
7             month    0.033571
18  day_of_year_sin    0.030658
13         campaign    0.015868
2         education    0.014599
1           marital    0.013883
16      balance_log    0.013044
11              day    0.011954
10          balance    0.011802
9               age    0.010807
0               job    0.010104
3           default    0.008371


In [ ]:
## CV 10 folds no previuous i z previous // n_trials = 100 /150 

In [ ]:
# n estimators 20000 i erly stopping

In [ ]:
# CATBOOST LIGHTXGBoost

In [ ]:

a=[
{'Model': 'XGBClassifier', 'ver':19, 'avg ROC-AUC':0.96788 , 'Public LB CV': 0.96924, 'Public LB no CV': 0.96878, 'Optuna n-trial': 30,  'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'Yes','del_columns':''               ,'opt_hyp_param':8 ,'New_columns':'' ,'Add_conditions': '', 'Best params': {'max_depth': 10, 'learning_rate': 0.088542375425159, 'n_estimators': 353, 'subsample': 0.7933680125492807, 'colsample_bytree': 0.665859932215261, 'gamma': 1.9591038948726576, 'reg_alpha': 4.135658777432998, 'reg_lambda': 3.0378978111339086}}, 
{'Model': 'XGBClassifier', 'ver':20, 'avg ROC-AUC':0.96787 , 'Public LB CV': 0.96942, 'Public LB no CV': 0.96914, 'Optuna n-trial': 30,  'CV folds-train': 5, 'Raw_data':'No', 'scale_pos_weight':'Yes','del_columns':'previous'       ,'opt_hyp_param':8 ,'New_columns':'' ,'Add_conditions': '', 'Best params':  {'max_depth': 9, 'learning_rate': 0.11410300715659989, 'n_estimators': 368, 'subsample': 0.907977121199552, 'colsample_bytree': 0.6014268962223818, 'gamma': 0.7012459904827175, 'reg_alpha': 4.361523501536162, 'reg_lambda': 4.294305690785547}},
{'Model': 'XGBClassifier', 'ver':21, 'avg ROC-AUC':0.96775 , 'Public LB CV': 0.96891, 'Public LB no CV': 0.96844, 'Optuna n-trial': 30,  'CV folds-train': 5, 'Raw_data':'No', 'scale_pos_weight':'Yes','del_columns':'previous'       ,'opt_hyp_param':8 ,'New_columns':'' ,'Add_conditions': 'enable_categorical=True', 'Best params':  {'max_depth': 8, 'learning_rate': 0.07733762308533498, 'n_estimators': 469, 'subsample': 0.9190881144681935, 'colsample_bytree': 0.7461531023839627, 'gamma': 2.06998675412871, 'reg_alpha': 1.3641260353935682, 'reg_lambda': 4.343504135311625}},
{'Model': 'XGBClassifier', 'ver':22, 'avg ROC-AUC':0.96765 , 'Public LB CV': 0.96886, 'Public LB no CV': 0.96848, 'Optuna n-trial': 30,  'CV folds-train': 5, 'Raw_data':'No', 'scale_pos_weight':'Yes','del_columns':'previous,pdays','opt_hyp_param':8 ,'New_columns':'no_previous_contact,pdays_cleaned,loan_plus_housing,deep_debt' ,'Add_conditions': 'days as category,', 'Best params':  {'max_depth': 9, 'learning_rate': 0.07135259874564061, 'n_estimators': 530, 'subsample': 0.8521429564138164, 'colsample_bytree': 0.7823863939520701, 'gamma': 2.9329829182260148, 'reg_alpha': 3.2901322430190185, 'reg_lambda': 1.268405294791835}},
{'Model': 'XGBClassifier', 'ver':23, 'avg ROC-AUC':0.96782 , 'Public LB CV': 0.96895, 'Public LB no CV': 0.96870, 'Optuna n-trial': 30,  'CV folds-train': 5, 'Raw_data':'No', 'scale_pos_weight':'Yes','del_columns':'previous,pdays','opt_hyp_param':8 ,'New_columns':'no_previous_contact,pdays_cleaned,loan_plus_housing' ,'Add_conditions': 'days as category', 'Best params': {'max_depth': 8, 'learning_rate': 0.12763052427108496, 'n_estimators': 411, 'subsample': 0.953989568332575, 'colsample_bytree': 0.6023438935073028, 'gamma': 0.019491768244553676, 'reg_alpha': 3.961328967768697, 'reg_lambda': 1.131707666650681} },
{'Model': 'XGBClassifier', 'ver':24, 'avg ROC-AUC':0.96756 , 'Public LB CV': 0.96867, 'Public LB no CV': 0.96831, 'Optuna n-trial': 30,  'CV folds-train': 5, 'Raw_data':'No', 'scale_pos_weight':'Yes','del_columns':'previous,pdays','opt_hyp_param':8 ,'New_columns':'no_previous_contact,pdays_cleaned,loan_plus_housing,no_count,unknown_count' ,'Add_conditions': 'days as category', 'Best params': {'max_depth': 7, 'learning_rate': 0.16686222138398568, 'n_estimators': 359, 'subsample': 0.9687425335082887, 'colsample_bytree': 0.8948829860260885, 'gamma': 0.3972938375538543, 'reg_alpha': 4.575416910972401, 'reg_lambda': 2.201098085025244} },
{'Model': 'XGBClassifier', 'ver':25, 'avg ROC-AUC':0.96792 , 'Public LB CV': 0.96900, 'Public LB no CV': 0.96883, 'Optuna n-trial': 30,  'CV folds-train': 5, 'Raw_data':'No' ,'scale_pos_weight':'Yes','del_columns':'previous,pdays','opt_hyp_param':8 ,'New_columns':'no_previous_contact,pdays_cleaned,no_count,unknown_count' ,'Add_conditions': 'days as category', 'Best params': {'max_depth': 10, 'learning_rate': 0.05576641173716932, 'n_estimators': 492, 'subsample': 0.7568779660915822, 'colsample_bytree': 0.6384212073745787, 'gamma': 2.0698430928317997, 'reg_alpha': 2.9165075287479447, 'reg_lambda': 2.040935203862332} },
{'Model': 'XGBClassifier', 'ver':26, 'avg ROC-AUC':0.96779 , 'Public LB CV': 0.96911, 'Public LB no CV':''     ,  'Optuna n-trial': 30,  'CV folds-train': 5, 'Raw_data':'No' ,'scale_pos_weight':'Yes','del_columns':'previous,pdays','opt_hyp_param':8 ,'New_columns':'no_previous_contact,pdays_cleaned','Add_conditions': '', 'Best params': {'max_depth': 10, 'learning_rate': 0.0832583908499121, 'n_estimators': 551, 'subsample': 0.8758220883852462, 'colsample_bytree': 0.6810585417529508, 'gamma': 2.389202320659961, 'reg_alpha': 0.9587487104983587, 'reg_lambda': 4.103575772005836} },
{'Model': 'XGBClassifier', 'ver':27, 'avg ROC-AUC':0.96786 , 'Public LB CV': 0.96901, 'Public LB no CV':''     ,  'Optuna n-trial': 30,  'CV folds-train': 5, 'Raw_data':'No' ,'scale_pos_weight':'Yes','del_columns':'previous'       ,'opt_hyp_param':8 ,'New_columns':'no_count' ,'Add_conditions':'', 'Best params': {'max_depth': 11, 'learning_rate': 0.07256654502258869, 'n_estimators': 264, 'subsample': 0.7252245807583018, 'colsample_bytree': 0.7225278113013671, 'gamma': 3.044364512727718, 'reg_alpha': 4.9552240198208874, 'reg_lambda': 3.7933495200093277} },
{'Model': 'XGBClassifier', 'ver':28, 'avg ROC-AUC':0.96787 , 'Public LB CV': 0.96913, 'Public LB no CV':''     ,  'Optuna n-trial': 30,  'CV folds-train': 5, 'Raw_data':'No' ,'scale_pos_weight':'Yes','del_columns':'previous'       ,'opt_hyp_param':8 ,'New_columns':'unknown_count' ,'Add_conditions':'', 'Best params': {'max_depth': 11, 'learning_rate': 0.05314481725365295, 'n_estimators': 485, 'subsample': 0.6616608475014232, 'colsample_bytree': 0.6827154636682595, 'gamma': 1.99699754127895, 'reg_alpha': 3.8063665621075478, 'reg_lambda': 1.325699284371194} },
{'Model': 'XGBClassifier', 'ver':30, 'avg ROC-AUC':0.96812 , 'Public LB CV': 0.96933, 'Public LB no CV':''     ,  'Optuna n-trial': 30,  'CV folds-train': 5, 'Raw_data':'No' ,'scale_pos_weight':'No' ,'del_columns':'previous'       ,'opt_hyp_param':8 ,'New_columns':'' ,'Add_conditions': '', 'Best params': {'max_depth': 11, 'learning_rate': 0.11206031547278855, 'n_estimators': 501, 'subsample': 0.6131323770348673, 'colsample_bytree': 0.6094568889243491, 'gamma': 2.043078715855179, 'reg_alpha': 2.0852138450654643, 'reg_lambda': 4.796903414902333} },
{'Model': 'XGBClassifier', 'ver':31, 'avg ROC-AUC':0.96819 , 'Public LB CV': 0.96955, 'Public LB no CV':''     ,  'Optuna n-trial': 30,  'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':8 ,'New_columns':'' ,'Add_conditions': '', 'Best params': {'max_depth': 9, 'learning_rate': 0.12075088743052882, 'n_estimators': 492, 'subsample': 0.8707220500188539, 'colsample_bytree': 0.6688353258198634, 'gamma': 0.7893885646435537, 'reg_alpha': 0.7438751764100405, 'reg_lambda': 3.4902318090320925} },
{'Model': 'XGBClassifier', 'ver':32, 'avg ROC-AUC':0.96849 , 'Public LB CV': 0.96944, 'Public LB no CV':''     ,  'Optuna n-trial': 30,  'CV folds-train': 10,'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':8 ,'New_columns':'' ,'Add_conditions': '', 'Best params': {'max_depth': 9, 'learning_rate': 0.06227423103688831, 'n_estimators': 556, 'subsample': 0.9298469326500857, 'colsample_bytree': 0.8895510541144902, 'gamma': 0.48209725496163736, 'reg_alpha': 3.796950975692023, 'reg_lambda': 1.6612550809713749} },
{'Model': 'XGBClassifier', 'ver':33, 'avg ROC-AUC':0.96801 , 'Public LB CV': 0.96903, 'Public LB no CV':''     ,  'Optuna n-trial': 30,  'CV folds-train': 10,'Raw_data':'No' ,'scale_pos_weight':'Yes','del_columns':'previous'       ,'opt_hyp_param':8 ,'New_columns':'' ,'Add_conditions': '', 'Best params': {'max_depth': 11, 'learning_rate': 0.04545133522176488, 'n_estimators': 481, 'subsample': 0.7684178974094136, 'colsample_bytree': 0.7986253830153623, 'gamma': 4.481651671654932, 'reg_alpha': 3.3989357087069094, 'reg_lambda': 0.557383341841934} },
{'Model': 'XGBClassifier', 'ver':34, 'avg ROC-AUC':0.97010 , 'Public LB CV': 0.97147, 'Public LB no CV':''     ,  'Optuna n-trial': 40,  'CV folds-train': 5, 'Raw_data':'No' ,'scale_pos_weight':'Yes','del_columns':'previous'       ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': '', 'Best params': {'max_depth': 11, 'learning_rate': 0.11017938521753814, 'n_estimators': 447, 'subsample': 0.7200594556282529, 'colsample_bytree': 0.7298687513204829, 'gamma': 3.7349463296256253, 'reg_alpha': 2.2870628091572853, 'reg_lambda': 0.3821213759830635, 'min_child_weight': 22, 'max_bin': 965, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':35, 'avg ROC-AUC':0.97018 , 'Public LB CV': 0.97150, 'Public LB no CV':''     ,  'Optuna n-trial': 40,  'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': '', 'Best params': {'max_depth': 10, 'learning_rate': 0.11419656400547268, 'n_estimators': 382, 'subsample': 0.8761249869278733, 'colsample_bytree': 0.9302155455418724, 'gamma': 0.922723932134517, 'reg_alpha': 1.0912230662828875, 'reg_lambda': 2.6136637861620295, 'min_child_weight': 23, 'max_bin': 923, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':36, 'avg ROC-AUC':0.97005 , 'Public LB CV': 0.97142, 'Public LB no CV':''     ,  'Optuna n-trial': 40,  'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'Yes','del_columns':''               ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': '', 'Best params': {'max_depth': 10, 'learning_rate': 0.08502933965618005, 'n_estimators': 492, 'subsample': 0.8475568692723069, 'colsample_bytree': 0.7937125702771929, 'gamma': 2.090595137285895, 'reg_alpha': 2.2649299117163197, 'reg_lambda': 4.582521266327681, 'min_child_weight': 18, 'max_bin': 850, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':37, 'avg ROC-AUC':0.97068 , 'Public LB CV': 0.97179, 'Public LB no CV':''     ,  'Optuna n-trial': 150, 'CV folds-train': 5, 'Raw_data':'No' ,'scale_pos_weight':'Yes','del_columns':'previous'       ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': 'n_estim, max_dep increased', 'Best params': {'max_depth': 13, 'learning_rate': 0.051597964533266545, 'n_estimators': 1199, 'subsample': 0.9486843342638015, 'colsample_bytree': 0.7012804860738411, 'gamma': 1.4669905642678631, 'reg_alpha': 3.8750691242340927, 'reg_lambda': 3.5772297879036534, 'min_child_weight': 17, 'max_bin': 1024, 'grow_policy': 'depthwise'} },
{'Model': 'XGBClassifier', 'ver':38, 'avg ROC-AUC':0.96976 , 'Public LB CV': 0.97147, 'Public LB no CV':''     ,  'Optuna n-trial': 40,  'CV folds-train': 5, 'Raw_data':'No' ,'scale_pos_weight':'Yes','del_columns':'previous'       ,'opt_hyp_param':11,'New_columns':'day_of_year_sin,day_of_year_cos' ,'Add_conditions': '', 'Best params': {'max_depth': 10, 'learning_rate': 0.12547508160977527, 'n_estimators': 558, 'subsample': 0.9603467732508381, 'colsample_bytree': 0.6775794698624192, 'gamma': 1.0653180442504713, 'reg_alpha': 3.49579539155107, 'reg_lambda': 3.985155368089634, 'min_child_weight': 12, 'max_bin': 787, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':40, 'avg ROC-AUC':0.97104 , 'Public LB CV': 0.97186, 'Public LB no CV':''     ,  'Optuna n-trial': 150, 'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 14, 'learning_rate': 0.02701453181796079, 'n_estimators': 1095, 'subsample': 0.8023378542955938, 'colsample_bytree': 0.6104508809024989, 'gamma': 0.45421103247113814, 'reg_alpha': 1.1316365943111237, 'reg_lambda': 1.6012566769586427, 'min_child_weight': 18, 'max_bin': 989} },
{'Model': 'XGBClassifier', 'ver':41, 'avg ROC-AUC':0.97160 , 'Public LB CV': 0.97248, 'Public LB no CV':''     ,  'Optuna n-trial':''  , 'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 14, 'learning_rate': 0.01624073742578193, 'n_estimators': 1632, 'subsample': 0.899429787551878, 'colsample_bytree': 0.6185268489803915, 'gamma': 0.36362513980290667, 'reg_alpha': 0.744367728380721, 'reg_lambda': 0.4628340619185789, 'min_child_weight': 9, 'max_bin': 1401, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':42, 'avg ROC-AUC':0.97189 , 'Public LB CV': 0.97316, 'Public LB no CV':''     ,  'Optuna n-trial':''  , 'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 19, 'learning_rate': 0.08091498981682245, 'n_estimators': 300, 'subsample': 0.8533765487098182, 'colsample_bytree': 0.6026759464047292, 'gamma': 0.0043501519621497176, 'reg_alpha': 3.4621137407155906, 'reg_lambda': 0.07698105946770128, 'min_child_weight': 1, 'max_bin': 2019, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':43, 'avg ROC-AUC':0.97188 , 'Public LB CV': 0.97282, 'Public LB no CV':''     ,  'Optuna n-trial':''  , 'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 17, 'learning_rate': 0.06519980970054881, 'n_estimators': 680, 'subsample': 0.8736025882716271, 'colsample_bytree': 0.875664321925202, 'gamma': 1.174374070219337, 'reg_alpha': 4.517748889976481, 'reg_lambda': 0.8719349977849177, 'min_child_weight': 5, 'max_bin': 1770, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':44, 'avg ROC-AUC':0.97201 , 'Public LB CV': 0.97299, 'Public LB no CV':''     ,  'Optuna n-trial':''  , 'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 18, 'learning_rate': 0.0708831450623464, 'n_estimators': 553, 'subsample': 0.8546933847917322, 'colsample_bytree': 0.7477903378445798, 'gamma': 1.196030349250854, 'reg_alpha': 3.387163897627949, 'reg_lambda': 0.8826864133067371, 'min_child_weight': 1, 'max_bin': 2022, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':45, 'avg ROC-AUC':0.97244 , 'Public LB CV': 0.97344, 'Public LB no CV':''     ,  'Optuna n-trial':''  , 'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 16, 'learning_rate': 0.0444106455843034, 'n_estimators': 2746, 'subsample': 0.8385441637974808, 'colsample_bytree': 0.8660799923129452, 'gamma': 0.6988818955694143, 'reg_alpha': 4.026337640164294, 'reg_lambda': 4.0256180856994215, 'min_child_weight': 3, 'max_bin': 1904, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':47, 'avg ROC-AUC':0.97248 , 'Public LB CV': 0.97356, 'Public LB no CV':''     ,  'Optuna n-trial': 120, 'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 8, 'learning_rate': 0.05003659706480022, 'n_estimators': 3599, 'subsample': 0.8429878804233633, 'colsample_bytree': 0.811904951081627, 'gamma': 0.4013365830244742, 'reg_alpha': 3.6000725691461786, 'reg_lambda': 0.23781092620605504, 'min_child_weight': 13, 'max_bin': 2442, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':48, 'avg ROC-AUC':0.97261 , 'Public LB CV': 0.97353, 'Public LB no CV':''     ,  'Optuna n-trial': 120, 'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 8, 'learning_rate': 0.030916714630792767, 'n_estimators': 3960, 'subsample': 0.8688930835089947, 'colsample_bytree': 0.8464859741187998, 'gamma': 0.03762546754890961, 'reg_alpha': 3.6580944723435667, 'reg_lambda': 0.00489088726961423, 'min_child_weight': 13, 'max_bin': 2127, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':49, 'avg ROC-AUC':0.97290 , 'Public LB CV': 0.97362, 'Public LB no CV':''     ,  'Optuna n-trial': 50 , 'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 8, 'learning_rate': 0.014826977628385373, 'n_estimators': 5465, 'subsample': 0.8043315478432289, 'colsample_bytree': 0.645060082872195, 'gamma': 0.20124387942031607, 'reg_alpha': 2.080967542693322, 'reg_lambda': 1.9436897009649072, 'min_child_weight': 5, 'max_bin': 2692, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':50, 'avg ROC-AUC':0.97290 , 'Public LB CV': 0.97381, 'Public LB no CV':''     ,  'Optuna n-trial': 50 , 'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 9, 'learning_rate': 0.027882762514711706, 'n_estimators': 4807, 'subsample': 0.8738983056484187, 'colsample_bytree': 0.7432234444382687, 'gamma': 0.6872650492187076, 'reg_alpha': 2.5245822386447596, 'reg_lambda': 0.7655196827452664, 'min_child_weight': 12, 'max_bin': 3023, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':51, 'avg ROC-AUC':0.97288 , 'Public LB CV': 0.97378, 'Public LB no CV':''     ,  'Optuna n-trial': 50 , 'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 10, 'learning_rate': 0.027455440776136326, 'n_estimators': 5934, 'subsample': 0.8465104721153287, 'colsample_bytree': 0.7026509223067279, 'gamma': 0.7470441137061977, 'reg_alpha': 3.1818845226707695, 'reg_lambda': 1.8736642668155106, 'min_child_weight': 12, 'max_bin': 3197, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':52, 'avg ROC-AUC':0.97288 , 'Public LB CV': 0.97373, 'Public LB no CV':''     ,  'Optuna n-trial': 50 , 'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 10, 'learning_rate': 0.028355231960286054, 'n_estimators': 4844, 'subsample': 0.9250167487880581, 'colsample_bytree': 0.7381558108905986, 'gamma': 0.5304502375744603, 'reg_alpha': 2.8679569811284966, 'reg_lambda': 2.78965614497557, 'min_child_weight': 11, 'max_bin': 2413, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':53, 'avg ROC-AUC':0.97283 , 'Public LB CV': 0.97383, 'Public LB no CV':''     ,  'Optuna n-trial': 20,  'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'Day_Year_sin, Day_Year_cos, balance_log, duration_log' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 20, 'learning_rate': 0.012775441668234806, 'n_estimators': 6911, 'subsample': 0.985977555279653, 'colsample_bytree': 0.6695853179005565, 'gamma': 0.14624951968603828, 'reg_alpha': 0.7772866902374195, 'reg_lambda': 4.016941662581965, 'min_child_weight': 25, 'max_bin': 2987, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':54, 'avg ROC-AUC':0.97296 , 'Public LB CV': 0.97393, 'Public LB no CV':''     ,  'Optuna n-trial': 20,  'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'Day_Year_sin, Day_Year_cos, balance_log, duration_log' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 5, 'learning_rate': 0.051544348346341265, 'n_estimators': 5986, 'subsample': 0.9406563987565996, 'colsample_bytree': 0.7032285244144026, 'gamma': 0.014005559892251412, 'reg_alpha': 4.47865750447716, 'reg_lambda': 2.5024030571202474, 'min_child_weight': 10, 'max_bin': 2801, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':55, 'avg ROC-AUC':0.97294 , 'Public LB CV': 0.97395, 'Public LB no CV':''     ,  'Optuna n-trial': 60,  'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'Day_Year_sin, Day_Year_cos, balance_log, duration_log' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 7, 'learning_rate': 0.053361175562108204, 'n_estimators': 4017, 'subsample': 0.9306723099635541, 'colsample_bytree': 0.7173136524092636, 'gamma': 0.3817608204886543, 'reg_alpha': 3.534578539601053, 'reg_lambda': 0.37926302495130126, 'min_child_weight': 11, 'max_bin': 3155, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':56, 'avg ROC-AUC':0.97298 , 'Public LB CV': 0.97399, 'Public LB no CV':''     ,  'Optuna n-trial': 60,  'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'Day_Year_sin, Day_Year_cos, balance_log, duration_log' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 6, 'learning_rate': 0.05315581672068095, 'n_estimators': 3947, 'subsample': 0.9341116539297873, 'colsample_bytree': 0.7121661138974171, 'gamma': 0.3250659086152249, 'reg_alpha': 3.110520372569841, 'reg_lambda': 2.5699454711304743, 'min_child_weight': 11, 'max_bin': 4074, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':57, 'avg ROC-AUC':0.97314 , 'Public LB CV': 0.97432, 'Public LB no CV':''     ,  'Optuna n-trial': 80,  'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'Day_Year_sin, Day_Year_cos, balance_log, duration_log' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 7, 'learning_rate': 0.050010678399715854, 'n_estimators': 4225, 'subsample': 0.9675105389521785, 'colsample_bytree': 0.6642729612066018, 'gamma': 0.06057145608089742, 'reg_alpha': 3.0245216337937118, 'reg_lambda': 0.6834653417762062, 'min_child_weight': 8, 'max_bin': 4403, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':58, 'avg ROC-AUC':0.97315 , 'Public LB CV': 0.97428, 'Public LB no CV':''     ,  'Optuna n-trial': 80,  'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'Day_Year_sin, Day_Year_cos, balance_log, duration_log' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 6, 'learning_rate': 0.04616333861262938, 'n_estimators': 5721, 'subsample': 0.9493894816676675, 'colsample_bytree': 0.6658104623618158, 'gamma': 0.21914049298728847, 'reg_alpha': 3.290083253769517, 'reg_lambda': 0.6407462194645503, 'min_child_weight': 9, 'max_bin': 4279, 'grow_policy': 'lossguide'} },
{'Model': 'XGBClassifier', 'ver':59, 'avg ROC-AUC':0.97314 , 'Public LB CV': 0.97427, 'Public LB no CV':''     ,  'Optuna n-trial': 80,  'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'Day_Year_sin, Day_Year_cos, balance_log, duration_log' ,'Add_conditions': 'wider range hyp-param', 'Best params': {'max_depth': 7, 'learning_rate': 0.04988569401318322, 'n_estimators': 3718, 'subsample': 0.9665374722246346, 'colsample_bytree': 0.661234616889833, 'gamma': 0.10614619651529983, 'reg_alpha': 3.1062652642283877, 'reg_lambda': 0.7947562715135358, 'min_child_weight': 8, 'max_bin': 4445, 'grow_policy': 'lossguide'} },
#{'Model': 'XGBClassifier', 'ver':53, 'avg ROC-AUC': , 'Public LB CV': , 'Public LB no CV':''     ,  'Optuna n-trial': 20,  'CV folds-train': 5, 'Raw_data':'Yes','scale_pos_weight':'No' ,'del_columns':''               ,'opt_hyp_param':11,'New_columns':'Day_Year_sin, Day_Year_cos, balance_log, duration_log' ,'Add_conditions': 'wider range hyp-param', 'Best params':  },
]




In [ ]:
rows = []
for d in a:
    params = d['Best params']
    row = {'ver': d['ver'], 'avg ROC-AUC': d['avg ROC-AUC']}
    row.update(params)
    rows.append(row)

df = pd.DataFrame(rows)

# Rysowanie wykresów
params_to_plot = [c for c in df.columns if c not in ['ver', 'avg ROC-AUC']]

for param in params_to_plot:
    plt.figure(figsize=(8,5))
    plt.scatter(df['ver'], df[param], c=range(len(df)), cmap='viridis', s=100)
    plt.plot(df['ver'], df[param], linestyle='--', alpha=0.6)
    for i, val in enumerate(df[param]):
        plt.text(df['ver'][i], val, f"{val:.3f}", fontsize=9, ha='center', va='bottom')
    plt.title(f"Hiperparametr: {param}")
    plt.xlabel("Wersja modelu")
    plt.ylabel(param)
    plt.colorbar(label="kolejność modelu")
    plt.show()

In [ ]:
df_a = pd.DataFrame(a)
df_a.iloc[:, :-1]

In [ ]:
import optuna
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

def optuna_search(n_trials=30, early_stopping_rounds=300):
    def objective(trial):
        params = {
            'max_depth': trial.suggest_int('max_depth', 3, 11),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
            'n_estimators': 10000,  # stała wartość, early stopping to ograniczy
            'eval_metric': 'auc',
            'use_label_encoder': False,
            'n_jobs': -1
        }

        scores = []
        for train_idx, val_idx in kf.split(X, y):
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

            model = XGBClassifier(**params,
                                  scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum())

            model.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                early_stopping_rounds=early_stopping_rounds,
                verbose=False
            )

            y_pred = model.predict_proba(X_val)[:, 1]
            auc = roc_auc_score(y_val, y_pred)
            scores.append(auc)

        return np.mean(scores)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    print("\n🎯 Najlepsze parametry Optuna:\n", study.best_params)
    print(f"🏆 Najlepszy ROC-AUC: {study.best_value:.5f}")

    # Trenowanie najlepszego modelu na całych danych
    best_params = study.best_params
    best_params['n_estimators'] = 10000
    best_params['eval_metric'] = 'auc'
    best_params['use_label_encoder'] = False
    best_params['n_jobs'] = -1

    final_model = XGBClassifier(**best_params,
                                scale_pos_weight=(y == 0).sum() / (y == 1).sum())

    final_model.fit(X, y, early_stopping_rounds=early_stopping_rounds, eval_set=[(X, y)], verbose=False)

    return final_model

In [ ]:
# Po zakończeniu fitowania z eval_set i early_stopping
best_n_estimators = model.best_iteration

# Trenujesz finalny model na całym zbiorze danych:
final_model = XGBClassifier(
    **study.best_params,
    n_estimators=best_n_estimators,
    use_label_encoder=False,
    eval_metric='auc',
    n_jobs=-1
)

final_model.fit(X, y)

 Uwagi:
To podejście łączy maksymalne wykorzystanie danych z optymalną liczbą iteracji.

Jest to powszechnie używana technika w realnych projektach.

Warto zapisać best_iteration po każdej foldzie podczas cross-validacji i np. uśrednić, jeśli chcesz mieć bardziej ogólny n_estimators.

# -----------------------------------------------------
# Optuna z early stopping i n_estimators = 20000
# -----------------------------------------------------

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv')

In [ ]:
# Check if new column can influence model learning process
train_df['month_as_num'] = train_df['month'].map({'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11, 'dec':12})
DAYS_IN_YEAR = 31 * 12  
# Number of the day in whole year
train_df['day_of_year'] = (train_df['month_as_num'] - 1) * 31 + train_df['day']
# Coding as cyclic data
train_df['day_of_year_sin'] = np.sin(2 * np.pi * train_df['day_of_year'] / DAYS_IN_YEAR)
train_df['day_of_year_cos'] = np.cos(2 * np.pi * train_df['day_of_year'] / DAYS_IN_YEAR)

train_df['balance_log'] = train_df['balance'].where(train_df['balance'] > -1, 0).apply(np.log1p)
train_df['duration_log'] = train_df['duration'].where(train_df['duration'] > -1, 0).apply(np.log1p)

In [ ]:
categorical_cols = [ 'job', 'marital','education','default','housing','loan','contact','month','poutcome']#,  'no_count', 'unknown_count' ]  # 'loan_plus_housing', <- wpisz swoje kolumny kategoryczne
numerical_cols = ['age','balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'balance_log', 'duration_log', 'day_of_year_sin', 'day_of_year_cos']#'day_of_year_sin', 'day_of_year_cos' ]#, 'previous' ]#'pdays']#, 'previous']    # <- wpisz swoje kolumny numeryczne
target_col = 'y'

In [ ]:
# Przygotowanie danych
df = train_df.copy()
X = df[categorical_cols + numerical_cols].copy()
y = df[target_col].copy()

# Label encoding dla kolumn kategorycznych
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# -----------------------------------------------------
# Funkcja trenowania z dużym n_estimators + early stopping
# -----------------------------------------------------
def train_model(X, y, params):
    scores = []
    importances = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

        model = XGBClassifier(
            **params,
            eval_metric='auc',
            use_label_encoder=False,
            early_stopping_rounds=500,
            n_jobs=-1
        )

        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            #early_stopping_rounds=300,
            verbose=False
        )

        y_pred = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)
        scores.append(auc)
        importances.append(model.feature_importances_)

        print(f"Fold {fold+1} ROC-AUC: {auc:.5f} | Trees used: {model.best_iteration}")
        print(f"[Train model] Fold {fold+1} - Best iteration: {model.best_iteration+1}")

    print(f"\nŚredni ROC-AUC: {np.mean(scores):.5f}")
    avg_importances = np.mean(importances, axis=0)

    importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': avg_importances
    }).sort_values(by='importance', ascending=False)

    print("\nFeature Importances:\n", importance_df)

    importance_df.plot.bar(x='feature', y='importance', legend=False, figsize=(10,5))
    plt.title("Średnia ważność cech (feature importance)")
    plt.show()



In [ ]:
def optuna_search(n_trials=30):
    def objective(trial):
        params = {
            'max_depth': trial.suggest_int('max_depth', 4, 12),
            'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.065),
            'subsample': trial.suggest_float('subsample',0.7, 0.95),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.95),
            'gamma': trial.suggest_float('gamma', 0, 1),
            'reg_alpha': trial.suggest_float('reg_alpha',2.5, 4.5),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 4),
            'min_child_weight': trial.suggest_int('min_child_weight', 8, 18),
            'max_bin': trial.suggest_int('max_bin', 2048, 5000),
            'grow_policy': 'lossguide',
            'n_estimators': 20000
        }

        scores = []
        fold_num = 1
        for train_idx, val_idx in kf.split(X, y):
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

            model = XGBClassifier(
                **params,
                eval_metric='auc',
                use_label_encoder=False,
                n_jobs=-1,
                early_stopping_rounds=500, #wstAW TUTAJ BY NIE bylo warningow
            )

            model.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                verbose=False
            )
            
            print(f"[Optuna] Fold {fold_num} - Best iteration: {model.best_iteration+1}")
            fold_num += 1

            y_pred = model.predict_proba(X_val)[:, 1]
            auc = roc_auc_score(y_val, y_pred)
            scores.append(auc)

        return np.mean(scores)

    study = optuna.create_study(direction='maximize')
    
    best_params_prev2={'max_depth': 6, 'learning_rate': 0.01, 'n_estimators': 20000, 'subsample': 0.9341116539297873, 'colsample_bytree': 0.7121661138974171, 'gamma': 0.3250659086152249, 'reg_alpha': 3.110520372569841, 'reg_lambda': 2.5699454711304743, 'min_child_weight': 11, 'max_bin': 4074, 'grow_policy': 'lossguide'}
    best_params_prev ={'max_depth': 6, 'learning_rate': 0.05315581672068095, 'n_estimators': 20000, 'subsample': 0.9341116539297873, 'colsample_bytree': 0.7121661138974171, 'gamma': 0.3250659086152249, 'reg_alpha': 3.110520372569841, 'reg_lambda': 2.5699454711304743, 'min_child_weight': 11, 'max_bin': 4074, 'grow_policy': 'lossguide'}  
    study.enqueue_trial(best_params_prev)
    study.enqueue_trial(best_params_prev2)
    
    study.optimize(objective, n_trials=n_trials)

    # Przekazanie stałych parametró (nie są zapisywane przez optunę do best_params)
    best_params = study.best_params.copy()
    best_params.update({'grow_policy': 'lossguide','n_estimators': 20000})

    print("\nNajlepsze parametry Optuna:\n", best_params)
    print(f"Najlepszy ROC-AUC: {study.best_value:.5f}")

    # Trening finalny z najlepszymi parametrami
    train_model(X, y, best_params)

In [ ]:
optuna_search(n_trials=60)

In [ ]:
Trial 1 finished with value: 0.970842402423133 and parameters: {'max_depth': 7, 'max_leaves': 1580, 'learning_rate': 0.0231469476935556, 'subsample': 0.8959315213545782, 'colsample_bytree': 0.7226883022153655, 'gamma': 0.5313913167805646, 'reg_alpha': 0.8914512027335614, 'reg_lambda': 1.2570717441067634, 'min_child_weight': 21, 'max_bin': 924, 'grow_policy': 'lossguide','n_estimators': 20000}


In [ ]:
Trial 7 finished with value: 0.9706474766357169 and parameters: {'max_depth': 7, 'learning_rate': 0.016481328062564214, 'subsample': 0.78538575960754, 'colsample_bytree': 0.8398589493683006, 'gamma': 1.1562617855827413, 'reg_alpha': 4.362527205707105, 'reg_lambda': 2.251389765899588, 'min_child_weight': 25, 'max_bin': 880, 'grow_policy': 'lossguide','n_estimators': 20000}
Fold 1 ROC-AUC: 0.97135 | Trees used: 16004
[Train model] Fold 1 - Best iteration: 16005
Fold 2 ROC-AUC: 0.97011 | Trees used: 16355
[Train model] Fold 2 - Best iteration: 16356
Fold 3 ROC-AUC: 0.97004 | Trees used: 14036
[Train model] Fold 3 - Best iteration: 14037
Fold 4 ROC-AUC: 0.97119 | Trees used: 16536
[Train model] Fold 4 - Best iteration: 16537
Fold 5 ROC-AUC: 0.97055 | Trees used: 16740
[Train model] Fold 5 - Best iteration: 16741

Średni ROC-AUC: 0.97065

Feature Importances:
       feature  importance
6     contact    0.201322
4     housing    0.182304
12   duration    0.148678
8    poutcome    0.133213
5        loan    0.068643
7       month    0.059143
14      pdays    0.056424
13   campaign    0.022101
1     marital    0.019592
2   education    0.019174
10    balance    0.018415
11        day    0.018264
15   previous    0.016265
9         age    0.015272
0         job    0.012285
3     default    0.008905

In [ ]:
best_params = {'max_depth': 7, 'learning_rate': 0.016481328062564214, 'subsample': 0.78538575960754, 'colsample_bytree': 0.8398589493683006, 'gamma': 1.1562617855827413, 'reg_alpha': 4.362527205707105, 'reg_lambda': 2.251389765899588, 'min_child_weight': 25, 'max_bin': 880}
best_params.update({'grow_policy': 'lossguide','n_estimators': 20000 })

best_params

In [ ]:
train_model(X, y, best_params)

# -----------------------------------------------------
# OOF target encoding osobno w każdym foldzie
# -----------------------------------------------------

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv')

In [ ]:
# Check if new column can influence model learning process
train_df['month_as_num'] = train_df['month'].map({'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11, 'dec':12})
DAYS_IN_YEAR = 31 * 12  
# Number of the day in whole year
train_df['day_of_year'] = (train_df['month_as_num'] - 1) * 31 + train_df['day']
# Coding as cyclic data
train_df['day_of_year_sin'] = np.sin(2 * np.pi * train_df['day_of_year'] / DAYS_IN_YEAR)
train_df['day_of_year_cos'] = np.cos(2 * np.pi * train_df['day_of_year'] / DAYS_IN_YEAR)

train_df['balance_log'] = train_df['balance'].where(train_df['balance'] > -1, 0).apply(np.log1p)
train_df['duration_log'] = train_df['duration'].where(train_df['duration'] > -1, 0).apply(np.log1p)

In [ ]:
# ==== WPROWADŹ SWOJE KOLUMNY ====
categorical_cols = [ 'job', 'marital','education','default','housing','loan','contact','month','poutcome']#,  'no_count', 'unknown_count' ]  # 'loan_plus_housing', <- wpisz swoje kolumny kategoryczne
numerical_cols = ['age','balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'balance_log', 'duration_log', 'day_of_year_sin', 'day_of_year_cos']#'day_of_year_sin', 'day_of_year_cos' ]#, 'previous' ]#'pdays']#, 'previous']    # <- wpisz swoje kolumny numeryczne
target_col = 'y'


In [ ]:
# ==== WCZYTAJ DANE ====
# train_df i test_df muszą być wcześniej wczytane

df = train_df.copy()
X = df[categorical_cols + numerical_cols].copy()
y = df[target_col].copy()

# ==== ENKODOWANIE DANYCH KATEGORYCZNYCH ==== Pomijać przy OOF Target encoding
# for col in categorical_cols:
#     le = LabelEncoder()
#     X[col] = le.fit_transform(X[col])

# for col in ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "poutcome"]:
#     X[col] = X[col].astype("category")

# ==== K-FOLD USTAWIENIA ====
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# ==== FUNKCJA TRENOWANIA MODELU ====
def train_model(X, y, params):
    scores = []
    importances = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

        model = XGBClassifier(
            **params,
            eval_metric='auc',
            #enable_categorical=True,
            use_label_encoder=False,
            n_jobs=-1,
         #   scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum()
        )
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)
        scores.append(auc)
        importances.append(model.feature_importances_)

        print(f"Fold {fold+1} ROC-AUC: {auc:.5f}")

    print(f"\nŚredni ROC-AUC: {np.mean(scores):.5f}")
    avg_importances = np.mean(importances, axis=0)

    importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': avg_importances
    }).sort_values(by='importance', ascending=False)

    print("\nFeature Importances:\n", importance_df)

    # Można też narysować
    importance_df.plot.bar(x='feature', y='importance', legend=False,  figsize=(10,5))
    plt.title("Średnia ważność cech (feature importance)")
    plt.show()

def oof_target_encode(X, y, categorical_cols, kf, smooth=20):
    X_encoded = X.copy()
    for col in categorical_cols:
        oof_col = np.zeros(len(X))
        global_mean = y.mean()

        for train_idx, val_idx in kf.split(X, y):
            df_train, df_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train = y.iloc[train_idx]

            stats = df_train.groupby(col)[y_train.name].agg(['mean','count'])
            stats['smooth'] = (stats['mean']*stats['count'] + global_mean*smooth) / (stats['count']+smooth)

            mapping = stats['smooth']
            oof_col[val_idx] = df_val[col].map(mapping).fillna(global_mean).values

        X_encoded[f"TE_{col}"] = oof_col.astype('float32')

    return X_encoded

In [ ]:
def fit_target_encoding(X, y, col, smooth=20):
    """
    Zwraca mapping: kategoria -> zakodowana wartość
    """
    global_mean = y.mean()
    df = pd.DataFrame({col: X[col], "_target": y})
    stats = df.groupby(col)["_target"].agg(["mean", "count"])
    stats["smooth"] = (stats["mean"]*stats["count"] + global_mean*smooth) / (stats["count"]+smooth)
    return stats["smooth"].to_dict(), global_mean


def transform_target_encoding(X, col, mapping, global_mean):
    return X[col].map(mapping).fillna(global_mean).astype("float32")


def oof_target_encode(X, y, categorical_cols, kf, smooth=20):
    """
    Zwraca X z nowymi kolumnami TE_*
    """
    X_encoded = X.copy()
    global_mean = y.mean()

    for col in categorical_cols:
        oof_col = np.zeros(len(X))
        
        for train_idx, val_idx in kf.split(X, y):
            X_tr, y_tr = X.iloc[train_idx], y.iloc[train_idx]
            X_val = X.iloc[val_idx]

            mapping, gmean = fit_target_encoding(X_tr, y_tr, col, smooth)
            oof_col[val_idx] = transform_target_encoding(X_val, col, mapping, gmean).values

        X_encoded[f"TE_{col}"] = oof_col

    return X_encoded

def encode_test(X_train, y_train, X_test, categorical_cols, smooth=20):
    """
    Transformacja testu na podstawie całego treningu
    """
    X_test_enc = X_test.copy()
    for col in categorical_cols:
        mapping, gmean = fit_target_encoding(X_train, y_train, col, smooth)
        X_test_enc[f"TE_{col}"] = transform_target_encoding(X_test, col, mapping, gmean)
    return X_test_enc

In [ ]:
def optuna_search(n_trials=30):
    def objective(trial):
        params = {
            'max_depth': trial.suggest_int('max_depth', 5, 20),#('max_depth', 7, 15),#3-11
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.08),
            'n_estimators': trial.suggest_int('n_estimators', 3000, 7000),#('n_estimators', 200, 1200),#200-600
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
# Additional parameters for optuna serch
            'min_child_weight': trial.suggest_int('min_child_weight', 5, 25),#('min_child_weight', 1, 28),
            'max_bin': trial.suggest_int('max_bin', 2048, 4096),#('max_bin', 256, 1024),
            'grow_policy': 'lossguide' #trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        }

        scores = []
        for train_idx, val_idx in kf.split(X, y):
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

            # OOF target encoding osobno w każdym foldzie
            #X_train_enc = oof_target_encode(X_train, y_train, categorical_cols, kf)
            #X_val_enc = oof_target_encode(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]), categorical_cols, kf).iloc[len(X_train):]
            # OOF encoding dla train + val
            X_train_enc = oof_target_encode(X_train, y_train, categorical_cols, kf)
            X_val_enc = encode_test(X_train, y_train, X_val, categorical_cols)
            
            te_cols = [f"TE_{c}" for c in categorical_cols]
            X_train_final = pd.concat([
                X_train_enc[numerical_cols].reset_index(drop=True),
                X_train_enc[te_cols].reset_index(drop=True)
            ], axis=1)
            
            X_val_final = pd.concat([
                X_val[numerical_cols].reset_index(drop=True),
                X_val_enc[te_cols].reset_index(drop=True)
            ], axis=1)
            
            X_train_final = X_train_final.astype("float32")
            X_val_final   = X_val_final.astype("float32")

            model = XGBClassifier(
                **params,
                eval_metric='auc',
                use_label_encoder=False,
                n_jobs=-1,
             #   scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum()
            )
            
            model.fit(X_train_final, y_train)
            y_pred = model.predict_proba(X_val_final)[:, 1]
            auc = roc_auc_score(y_val, y_pred)
            scores.append(auc)

        return np.mean(scores)


    study = optuna.create_study(direction='maximize')
    #Dodanie najlepszych param z przeszłości
    # best_params_prev ={'max_depth': 8, 'learning_rate': 0.05003659706480022, 'n_estimators': 3599, 'subsample': 0.8429878804233633, 'colsample_bytree': 0.811904951081627, 'gamma': 0.4013365830244742, 'reg_alpha': 3.6000725691461786, 'reg_lambda': 0.23781092620605504, 'min_child_weight': 13, 'max_bin': 2442, 'grow_policy': 'lossguide'}  
    # study.enqueue_trial(best_params_prev)
    study.optimize(objective, n_trials=n_trials)

    # Przekazanie stałych parametró (nie są zapisywane przez optunę do best_params)
    best_params = study.best_params.copy()
    best_params.update({'grow_policy': 'lossguide'})

    print("\nNajlepsze parametry Optuna:\n", best_params)
    print(f"Najlepszy ROC-AUC: {study.best_value:.5f}")

    # Trenowanie najlepszego modelu na koniec
    #train_model(X, y, best_params)

In [ ]:
optuna_search(n_trials=10)